In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Function to scrape the lyrics from a single song link
def scrape_lyrics(song_url):
    try:
        response = requests.get(song_url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the song title
        title_tag = soup.find('h1') or soup.find('h2')  # Adjust based on the website's structure
        title = title_tag.get_text(strip=True) if title_tag else "Unknown Title"

        # Extract the lyrics (adjust the class/id/tag based on the website's structure)
        lyrics_div = soup.find('div', {'class': 'post-body'})  # Example for Blogger-based sites
        lyrics = lyrics_div.get_text("\n", strip=True) if lyrics_div else "Lyrics not found"

        return {"url": song_url, "title": title, "lyrics": lyrics}
    except Exception as e:
        print(f"Error scraping {song_url}: {e}")
        return {"url": song_url, "title": "Error", "lyrics": "Error"}

# Function to read song links from a CSV file
def read_song_links(filename="all_song_links.csv"):
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        return [row['url'] for row in reader]

# Function to save scraped lyrics into a CSV file
def save_lyrics_to_csv(lyrics_data, filename="scraped_lyrics.csv"):
    filepath = f"/mnt/data/{filename}"  # Save to a location accessible for download
    with open(filepath, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["url", "title", "lyrics"])
        writer.writeheader()
        writer.writerows(lyrics_data)
    print(f"Saved {len(lyrics_data)} songs with lyrics to {filepath}")
    return filepath

# Main script to scrape lyrics from all song links
def main():
    song_links = read_song_links()  # Read the list of song URLs
    all_lyrics = []

    for i, song_url in enumerate(song_links, 1):
        print(f"Scraping lyrics for song {i}/{len(song_links)}: {song_url}")
        song_data = scrape_lyrics(song_url)
        all_lyrics.append(song_data)
        time.sleep(2)  # Add delay to avoid overloading the server

    # Save lyrics to CSV
    csv_filepath = save_lyrics_to_csv(all_lyrics)
    return csv_filepath

if __name__ == "__main__":
    csv_file = main()
    print(f"CSV file saved at: {csv_file}")


Scraping lyrics for song 1/2744: https://sahityasudhe.blogspot.com/2012/01/blog-post_2985.html
Scraping lyrics for song 2/2744: https://sahityasudhe.blogspot.com/2012/01/blog-post_6865.html
Scraping lyrics for song 3/2744: https://sahityasudhe.blogspot.com/search/label/%E0%B2%8E%E0%B2%9A%E0%B3%8D.%20%E0%B2%8E%E0%B2%B8%E0%B3%8D.%20%E0%B2%B5%E0%B3%86%E0%B2%82%E0%B2%95%E0%B2%9F%E0%B3%87%E0%B2%B6%E0%B2%AE%E0%B3%82%E0%B2%B0%E0%B3%8D%E0%B2%A4%E0%B2%BF?updated-max=2013-01-10T21:48:00%2B05:30&max-results=5&start=5&by-date=false
Scraping lyrics for song 4/2744: https://sahityasudhe.blogspot.com/2014/02/blog-post_9158.html
Scraping lyrics for song 5/2744: https://sahityasudhe.blogspot.com/2012/01/blog-post_721.html
Scraping lyrics for song 6/2744: https://sahityasudhe.blogspot.com/2013/01/1982.html
Scraping lyrics for song 7/2744: https://sahityasudhe.blogspot.com/2012/01/blog-post_7100.html
Scraping lyrics for song 8/2744: https://sahityasudhe.blogspot.com/2012/01/blog-post_6787.html
Scraping l

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of song URLs
song_links = ["https://sahityasudhe.blogspot.com/search?updated-max=2017-08-28T12:02:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A7%E0%B2%BE%E0%B2%B0%E0%B2%BE%E0%B2%B5%E0%B2%BE%E0%B2%B9%E0%B2%BF?updated-max=2020-08-25T09:02:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5659.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_39.html",
"https://sahityasudhe.blogspot.com/2012/06/1996_2729.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7802.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AD%E0%B2%95%E0%B3%8D%E0%B2%A4%E0%B2%BF%E0%B2%97%E0%B3%80%E0%B2%A4%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/1965.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2645.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%86%E0%B2%A8%E0%B2%82%E0%B2%A6%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4973.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B3%81%E0%B2%B0%E0%B2%82%E0%B2%A6%E0%B2%B0%20%E0%B2%A6%E0%B2%BE%E0%B2%B8?updated-max=2013-01-18T21:35:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8939.html",
"https://sahityasudhe.blogspot.com/2017/09/bhagyada-lakshmi-baramma.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%8A%E0%B2%82%E0%B2%AC%E0%B2%BF%E0%B2%B8%E0%B2%BF%E0%B2%B2%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2985.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%86%E0%B2%95%E0%B2%BE%E0%B2%B6%E0%B3%8D?updated-max=2012-01-21T07:56:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B5%E0%B2%BF%E0%B2%9C%E0%B2%AF%E0%B2%A8%E0%B2%BE%E0%B2%B0%E0%B2%B8%E0%B2%BF%E0%B2%82%E0%B2%B9?updated-max=2013-01-10T21:41:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2743.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B2%E0%B3%81%E0%B2%B8%E0%B2%BF%E0%B2%AF%E0%B2%BE?updated-max=2012-11-06T20:07:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%97%E0%B2%BE%E0%B2%A8%20%E0%B2%AC%E0%B2%9C%E0%B2%BE%E0%B2%A8?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8921.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8426.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2412.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4182.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-21T08:02:00%2B05:30&max-results=20&start=422&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/2006_243.html",
"https://sahityasudhe.blogspot.com/2012/06/2006.html",
"https://sahityasudhe.blogspot.com/2013/01/1997.html",
"https://sahityasudhe.blogspot.com/2012/01/1963.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_58.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%B5%E0%B2%BF%E0%B2%B0%E0%B2%A4%E0%B3%8D%E0%B2%A8%20%E0%B2%95%E0%B2%BE%E0%B2%B3%E0%B2%BF%E0%B2%A6%E0%B2%BE%E0%B2%B8?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1083.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7931.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%A3%E0%B3%8D%E0%B2%A3%E0%B2%BF%E0%B2%A8%20%E0%B2%A6%E0%B3%8B%E0%B2%A3%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%8E%E0%B2%AE%E0%B3%8D.%20%E0%B2%8E%E0%B2%A8%E0%B3%8D.%20%E0%B2%B5%E0%B3%8D%E0%B2%AF%E0%B2%BE%E0%B2%B8%20%E0%B2%B0%E0%B2%BE%E0%B2%B5%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4182.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_16.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_9751.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_937.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8910.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2886.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_70.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5922.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2758.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2475.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9424.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_27.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_50.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2333.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2851.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8805.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2609.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_325.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2886.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%A8%E0%B3%81%E0%B2%B0%E0%B2%BE%E0%B2%97%20%E0%B2%85%E0%B2%B0%E0%B2%B3%E0%B2%BF%E0%B2%A4%E0%B3%81?updated-max=2012-01-23T19:23:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/06/2006_12.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_90.html",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu.html",
"https://sahityasudhe.blogspot.com/2017/09/nammamma-sharade.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_39.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1635.html",
"https://sahityasudhe.blogspot.com/2017/09/gajavadana-beduve.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%B6%E0%B3%8D%E0%B2%B5%E0%B2%AE%E0%B3%87%E0%B2%A7?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A4%E0%B2%BE%E0%B2%9C%E0%B3%8D%20%E0%B2%AE%E0%B2%B9%E0%B2%B2%E0%B3%8D?updated-max=2012-01-20T08:01:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2985.html",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2954.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_63.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2019-07-25T16:16:00%2B05:30&max-results=10",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8426.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%A8%E0%B2%B8%E0%B2%B2%E0%B3%82%20%E0%B2%A8%E0%B3%80%E0%B2%A8%E0%B3%86%20%E0%B2%AE%E0%B2%A8%E0%B2%B8%E0%B2%B2%E0%B3%81%20%E0%B2%A8%E0%B3%80%E0%B2%A8%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_49.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_25.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7100.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%82%E0%B2%A4?updated-max=2012-01-23T19:21:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_89.html",
"https://sahityasudhe.blogspot.com/2013/01/2011_9.html",
"https://sahityasudhe.blogspot.com/2012/01/1963.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_771.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_58.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7024.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7132.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2737.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3948.html",
"https://sahityasudhe.blogspot.com/2012/01/paramatma.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_16.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8141.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5543.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_97.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8910.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post.html",
"https://sahityasudhe.blogspot.com/2012/06/2004_7130.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4464.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B5%E0%B2%BF.%20%E0%B2%A8%E0%B2%BE%E0%B2%97%E0%B3%87%E0%B2%82%E0%B2%A6%E0%B3%8D%E0%B2%B0%20%E0%B2%AA%E0%B3%8D%E0%B2%B0%E0%B2%B8%E0%B2%BE%E0%B2%A6%E0%B3%8D?updated-max=2013-01-18T21:29:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8910.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7024.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%B0%E0%B2%AA%E0%B2%82%E0%B2%9A?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2541.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_79.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2609.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_325.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8805.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_937.html",
"https://sahityasudhe.blogspot.com/2012/06/2006_12.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1306.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B2%B5%E0%B2%97%E0%B3%8D%E0%B2%B0%E0%B2%B9?updated-max=2012-01-20T08:10:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_91.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%BF.%E0%B2%B2%E0%B2%82%E0%B2%95%E0%B3%87%E0%B2%B6?updated-max=2014-03-16T15:21:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_370.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_90.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2743.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BF%E0%B2%AE%E0%B3%8D%E0%B2%AE%E0%B2%BF%20%E0%B2%97%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B3%81?updated-max=2013-01-09T21:28:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_574.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B3%81%E0%B2%B2%E0%B2%B5%E0%B2%A7%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/2011_143.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_10.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9090.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%83%E0%B2%A6%E0%B2%AF%E0%B2%B6%E0%B2%BF%E0%B2%B5?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9581.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%82%E0%B2%97%E0%B3%8D%E0%B2%B2%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/09/nammamma-sharade.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7796.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4059.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2954.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-16T07:58:00%2B05:30&max-results=20&start=202&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_63.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_70.html",
"https://sahityasudhe.blogspot.com/2017/09/satyavantara-sangaviralu.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_94.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_9473.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9102.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6378.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_6449.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1089.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BF.%20%E0%B2%8E%E0%B2%B8%E0%B3%8D.%20%E0%B2%B6%E0%B2%BF%E0%B2%B5%E0%B2%B0%E0%B3%81%E0%B2%A6%E0%B3%8D%E0%B2%B0%E0%B2%AA%E0%B3%8D%E0%B2%AA?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B2%B5%E0%B2%B3%E0%B3%8D%E0%B2%B3%E0%B2%BF%E0%B2%AF%20%E0%B2%A4%E0%B3%8B%E0%B2%9F?updated-max=2012-11-06T20:11:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5044.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_370.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_27.html",
"https://sahityasudhe.blogspot.com/2013/01/2006_4334.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_243.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%AF%E0%B2%B8%E0%B2%BF%E0%B2%82%E0%B2%B9?updated-max=2013-01-09T21:26:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/06/1996.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AC%E0%B2%BF.%20%E0%B2%8E%E0%B2%82.%20%E0%B2%B6%E0%B3%8D%E0%B2%B0%E0%B3%80%E0%B2%95%E0%B2%82%E0%B2%A0%E0%B2%AF%E0%B3%8D%E0%B2%AF?updated-max=2013-01-18T21:22:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1981_3716.html",
"https://sahityasudhe.blogspot.com/2012/01/1990_17.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2645.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_9809.html",
"https://sahityasudhe.blogspot.com/2012/01/1963_3599.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5659.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9581.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8805.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7525.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1635.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%B2%E0%B2%BE%E0%B2%B5%E0%B2%BF%E0%B2%A6?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_15.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AD%E0%B2%95%E0%B3%8D%E0%B2%A4%E0%B2%BF%E0%B2%97%E0%B3%80%E0%B2%A4%E0%B3%86?updated-max=2017-09-06T10:15:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_48.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5439.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2743.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8884.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2462.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8939.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%A8%E0%B2%95%20%E0%B2%A6%E0%B2%BE%E0%B2%B8%E0%B2%B0%E0%B3%81?updated-max=2012-01-20T08:07:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7382.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_10.html",
"https://sahityasudhe.blogspot.com/2012/01/1986_17.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6279.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9631.html?showComment=1353688172997#c419858439249332768",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-10T21:23:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-04-25T23:18:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4919.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_27.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A6%E0%B3%8A%E0%B2%A1%E0%B3%8D%E0%B2%A1%E0%B2%B0%E0%B2%82%E0%B2%97%E0%B3%86%20%E0%B2%97%E0%B3%8C%E0%B2%A1?updated-max=2013-01-18T21:31:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1862.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_70.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3452.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4201.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7382.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8483.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2263.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_94.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AF%E0%B3%81%E0%B2%97%E0%B2%AA%E0%B3%81%E0%B2%B0%E0%B3%81%E0%B2%B7?updated-max=2012-01-21T07:57:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/09/krishna-nee-begane-baaro.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_98.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5218.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_72.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_20.html",
"https://sahityasudhe.blogspot.com/2012/11/2012_6.html",
"https://sahityasudhe.blogspot.com/2012/01/1981.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%87%E0%B2%AC%E0%B3%8D%E0%B2%AC%E0%B2%A8%E0%B2%BF%20%E0%B2%95%E0%B2%B0%E0%B2%97%E0%B2%BF%E0%B2%A4%E0%B3%81?updated-max=2012-01-23T19:18:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_3.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_442.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_9704.html",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu_40.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%B0%E0%B2%AE%E0%B2%BE%E0%B2%A4%E0%B3%8D%E0%B2%AE?&max-results=5",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_44.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_243.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5218.html",
"https://sahityasudhe.blogspot.com/2017/08/kirik-party-belageddu-lyrics.html"
"https://sahityasudhe.blogspot.com/2012/04/2009_9809.html",
"https://sahityasudhe.blogspot.com/2012/01/1963_3599.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_60.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7423.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7741.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7525.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2542.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6378.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1635.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1879.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1809.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9344.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_4.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2462.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-09T21:26:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4090.html",
"https://sahityasudhe.blogspot.com/2012/01/1982.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%BE%E0%B2%A8%E0%B2%B8%20%E0%B2%B8%E0%B2%B0%E0%B3%8B%E0%B2%B5%E0%B2%B0?updated-max=2012-01-20T07:55:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_82.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_7691.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2356.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8420.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3521.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7093.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2915.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2541.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1476.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9102.html",
"https://sahityasudhe.blogspot.com/2017/08/neenire-saniha-kirik-party-song-lyrics.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B3%82..%E0%B2%9F%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_243.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B2%82%E0%B2%97%E0%B2%AE?&max-results=5",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-24T12:20:00%2B05:30&max-results=20&start=107&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/2011_9.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B2%A8%E0%B2%A6%20%E0%B2%97%E0%B3%8A%E0%B2%82%E0%B2%AC%E0%B3%86?updated-max=2012-11-06T20:10:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%83%E0%B2%A6%E0%B2%AF%E0%B2%B6%E0%B2%BF%E0%B2%B5?updated-max=2013-01-16T07:53:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_442.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_34.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%AF%E0%B2%82%E0%B2%A4%E0%B3%8D%20%E0%B2%95%E0%B2%BE%E0%B2%AF%E0%B3%8D%E0%B2%95%E0%B2%BF%E0%B2%A3%E0%B2%BF?updated-max=2017-08-23T10:23:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1089.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8527.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B0%E0%B2%BE%E0%B2%AF%E0%B2%B0%E0%B3%81%20%E0%B2%AC%E0%B2%82%E0%B2%A6%E0%B2%B0%E0%B3%81%20%E0%B2%AE%E0%B2%BE%E0%B2%B5%E0%B2%A8%20%E0%B2%AE%E0%B2%A8%E0%B3%86%E0%B2%97%E0%B3%86?updated-max=2012-01-23T19:16:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/kirik-party-belageddu-lyrics.html"

]

def classify_rasa(lyrics):
    rasa_keywords = {
        "Devotional": ["ದೇವ", "ಭಕ್ತಿ", "ಪೂಜೆ", "ಹರಿಯೆ", "ಶರಣು", "ರಾಮ", "ಕೃಷ್ಣ", "ವಿಷ್ಣು", "ಪರಮಾತ್ಮ", "ಆರಾಧನೆ", "ಧರ್ಮ", "ಪರಮೇಶ್ವರ", "ಭಕ್ತರು", "ಅರ್ಚನೆ", "ಗೋವಿಂದ", "ಶಿವ", "ದೇವತೆ", "ಅಷ್ಟೋತ್ತರ", "ವಂದನೆ", "ಪೂಜಾರಿ", "ಪವಿತ್ರ", "ಸಂಸ್ಕಾರ", "ಆರಾಧನೆ", "ಸಾಧನೆ", "ಯೋಗ", "ಧ್ಯಾನ", "ಹನುಮಾನ್", "ಪರಮಭಕ್ತ", "ಗುರು", "ದೀಕ್ಷೆ", "ಪ್ರಾರ್ಥನೆ", "ಸಾನ್ನಿಧ್ಯ", "ಶಕ್ತಿ", "ಅಧ್ಯಾತ್ಮ", "ಆತ್ಮದರ್ಶನ", "ಪೂಜಿತ", "ಸಾಗರ", "ಪಾರಮಾರ್ಥ", "ಅವತಾರ", "ಪುನರ್ಜನ್ಮ", "ಬ್ರಹ್ಮ", "ಮೋಕ್ಷ", "ಪುನೀತ", "ಜಪ", "ಸ್ತೋತ್ರ", "ಅಂತ್ಯಕಥೆ", "ಆಧ್ಯಾತ್ಮಿಕ", "ಅಮೃತ", "ಚಕ್ರ"],
        "Patriotic": ["ದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ", "ಜೈಹಿಂದ್", "ಸಮರ್ಪಣೆ", "ಭಾರತ", "ಧ್ವಜ", "ಸ್ವಾಭಿಮಾನ", "ವೀರ", "ರಾಜ್ಯ", "ಏಕತೆ", "ರಾಷ್ಟ್ರ", "ಬಾಳಿದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ ಹೋರಾಟ", "ಭಾರತ ಮಾತಾ", "ಜೈ ಭಾರತ", "ಭಾರತೀಯ", "ಹೋರಾಟಗಾರ", "ಬಸವಣ್ಣ", "ಸ್ಪರ್ಧೆ", "ವೀರಸೇನೆ", "ಸಮಾನತೆ", "ಪ್ರಜಾಸತ್ತಾತ್ಮಕ", "ಜಾತಿ", "ರಾಷ್ಟ್ರಪ್ರೇಮ", "ಸೈನಿಕ", "ಸಾಮರಸ್ಯ", "ಹಕ್ಕು", "ವಿದ್ಯಾರ್ಥಿ", "ಸಂಘಟನೆ", "ಸಂಭ್ರಮ", "ಬಿಗ್ ಡೇ", "ಅಜಯ್ ಭಾರತ", "ದೇಶಭಕ್ತ", "ಪಾಲ್ಗೊಂಡರು", "ಹೋರಾಟ", "ಆಂದೋಲನ", "ಸಂಬಂಧ", "ಹೈದರಾಬಾದ್", "ಗೌರವ", "ಆತ್ಮವಿಶ್ವಾಸ", "ಧೈರ್ಯ", "ಅಖಂಡ ಭಾರತ", "ಪರಾಕ್ರಮ", "ರಕ್ಷಕ", "ಪರಮವೀರ", "ಪರಿಪೂರ್ಣ", "ಶೌರ್ಯ", "ವಿಜಯ", "ವಿಜಯೋತ್ಸವ"],
        "Folk": ["ಜನಪದ", "ಹೊಲವು", "ಕುಟುಕು", "ಗ್ರಾಮೀಣ", "ಬಾಲಪಾಟ", "ಕುಣಿತ", "ನಾಡು", "ಹಬ್ಬ", "ಹಳ್ಳಿ", "ಕೃತನೆ", "ಹೆಜ್ಜೆ", "ಹೊರಾಟ", "ಬಾವಲಿ", "ದೋಣಿ", "ಮುದ್ದು", "ಬಡವರ", "ಬೂದಿ", "ಮಳಗು", "ಪಾದ", "ಬದನೆ", "ಹಳ್ಳಿಗುಡಿ", "ಕಾಯ್ಕಳು", "ಕಬ್ಬಿಣ", "ಕೃಷಿ", "ಬಟ್ಟಲು", "ಹಾದಿ", "ಬೀದಿ", "ಅಂಗಡಿ", "ರಸಗೆಜೆ", "ಗೊಡ್ಡಲು", "ಅರವಟ್ಟೆ", "ಅಕ್ಕಸಾಲಿಗೆ", "ಆಡುಗೊಳಲು", "ಅಕ್ಕಪಕ್ಕ", "ಹೂವಿನ ಹಾರ", "ಮಡಕೆ", "ಮೂಡಲಗೇರಿ", "ಮಕ್ಕಳಾಟ", "ನೆಲದಾಟ", "ಮಾಲಿಗೆ", "ಚಪ್ಪಲಿ", "ಬಸವಣ", "ದೀಪಾಲಂಕಾರ", "ಗಂಗಾವತಿ", "ಕಂಭಳ", "ಹೆಜ್ಜೆಗುಂಡಿ", "ಮರಹರಕೆ", "ಚಂದ್ರಹಾರ", "ಪಡುಗೆಯಾಟ", "ಬೂತಕೋಲ"],
        "Happy": ["ನಗುವು", "ಸಂತೋಷ", "ಹರ್ಷ", "ಆನಂದ", "ಉಲ್ಲಾಸ", "ಮೋಹಕ", "ಸಂತೃಪ್ತಿ", "ಹರ್ಷೋಲ್ಲಾಸ", "ಉತ್ಸವ", "ಸುವಾಸನೆ", "ಶ್ರದ್ಧೆ", "ಪ್ರಸನ್ನತೆ", "ಪ್ರಕೃತಿ", "ಶಕ್ತಿಮಂತ್ರ", "ಆಲಾಪನೆ", "ಆಹ್ಲಾದ", "ಚಿರಂಜೀವಿ", "ಜೋಶ್", "ಸ್ಫೂರ್ತಿ", "ಮೋಜು", "ಹಾಸ್ಯ", "ಪ್ರಜ್ಞೆ", "ಸಂಬ್ರಮ", "ಆಕರ್ಷಣ", "ಮಹೋತ್ಸವ", "ಮಹಿಮಾ", "ಪ್ರಾರ್ಥನೆ", "ಆನಂದಿತ", "ಮೋಜುಗಾಡು", "ಚಲನೆ", "ಮೋಹನೆ", "ಅಮೃತ", "ಆನಂದಧಾರೆ", "ಹಬ್ಬದಚಟುವಟಿಕೆಗಳು", "ಸಂಗೀತ", "ಜಶ್ನ್", "ಅದ್ಭುತ", "ವಿಶೇಷ", "ಹೃತ್ಪೂರ್ವಕ", "ಆರಂಭ", "ಆನಂದೋತ್ಸವ", "ಪ್ರಸನ್ನಮುಖ", "ಸುಖಪೂರ್ಣ", "ಹಾಸ್ಯಚಟುವಟಿಕೆ", "ಸ್ನೇಹಪೂರ್ಣ", "ಚಿರಪರಿಚಿತ", "ಪ್ರೋತ್ಸಾಹ", "ಸಿರಿವಂತಿಕೆ", "ಸುದೀರ್ಘ"],
        "Sad": ["ದುಃಖ", "ಕಣ್ಣೀರು", "ನೋವು", "ಬಾಳು", "ವಿಚಾರ", "ಕಳವಳ", "ಮರುಳು", "ಪೀಡೆ", "ಆರತಿ", "ವಿಪತ್ತು", "ತೀವ್ರ ನೋವು", "ಬಿರುಗಾಳಿ", "ಮರಣ", "ಆಲೋಚನೆ", "ಪರಿಶುದ್ಧತೆ", "ಬೇಸರ", "ಪಾತಕ", "ನಷ್ಟ", "ವಿರಹ", "ಆರಾಧನೆ", "ಆಶ್ರು", "ಬದ್ಧ", "ಮೋಹಪಾಶ", "ಅನಾಥ", "ವಿದಾಯ", "ಆಳ್ವಿಕೆ", "ಅಪಘಾತ", "ಶೋಕ", "ಅಪರಾಧ ಭಾವನೆ", "ಪತಿತ", "ಪತನ", "ಹೃದಯವೇದನೆ", "ವಿಚಲನೆ", "ಆರಂಬವೇದನೆ", "ಅಪಮಾನ", "ಅಸಹಾಯಕತೆ", "ಬಿರುಗಾಳಿ", "ಅಪಶ್ರಯ", "ಹೀನತೆಗೆ", "ದ್ವೇಷ", "ಅಧ್ಯಾತ್ಮ ಪೀಡೆ", "ಮರಮಾತು", "ಅಪಸ್ಮೃತಿ", "ಅನಿರೀಕ್ಷಿತ ಪೀಡೆ", "ಹೀನಗತಿ", "ಮೂಢತನ", "ಅಪಮಾನಾತ್ಮಕ", "ಬಿದಿರಹಿತ", "ಹೀನಭಾವ"],
        "Fear": ["ಭಯ", "ಗಾಬರಿ", "ಅಜ್ಞಾನ", "ವಿಕಂಪ", "ಭೀತಿ", "ಆತಂಕ", "ಹೆದರಿಕೆ", "ಭ್ರಾಂತಿ", "ಕಂಪನ", "ಅಪಾಯ", "ಆಘಾತ", "ಅಪಘಾತ", "ಅಪರೂಪ", "ಹಿಡಿದುಕೊಳ್ಳು", "ಅಪರಿಚಿತ", "ಪೀಡೆ", "ಅಪರಾಧ", "ಆಸ್ಪದ", "ಬಿಥ್", "ಅಪಾಯಕಾರಿ", "ಆಕಸ್ಮಿಕ", "ಅಪಾಯಕಾರಕ", "ಅಪಾಯಕಾರಿ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ", "ಅಪಹರಣ", "ಅಪಹಾಸ್ಯ", "ಅನಿಶ್ಚಿತತೆ", "ಅಪಸ್ವರ", "ಅಪಪ್ರಚಾರ", "ಅಪಾಧರಿತ", "ಅಪರಿಚಿತ", "ಅಪಾಯಸೂಚಕ", "ಹಿಂಸೆ", "ಅಪಲೋಚನೆ", "ಅಪರೂಪವಾದ", "ಅಪರಾಧಿ", "ಅಪಮಾನದಾಯಕ", "ಅಪಾಯಕಾರಿ ದಾರಿ", "ಅಪಾಯದ ಕ್ಷಣ", "ಭೀತಿಗೊಳಗಾಗು", "ಆತಂಕಭರಿತ", "ಆಘಾತ ಭರಿತ", "ಅಪಮಾನಿತ", "ಭಯಾನಕ ದೃಶ್ಯ", "ಅಪಯಶಸ್ವಿ", "ಅಪಾಯಕಾರಕ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ ಘಟನೆ", "ಅಪಹೃತ"],
        "Love": ["ಪ್ರೇಮ", "ಮನಸೆ", "ನೀನು", "ನನ್ನ", "ಪ್ರೀತಿ", "ಸ್ನೇಹ", "ಅನುರಾಗ", "ಆಕರ್ಷಣೆ", "ಮಧುರ", "ಅಭಿಮಾನ", "ರೋಮ್ಯಾಂಸ್", "ವೈಭವ", "ಸ್ನೇಹಭಾವ", "ಮೋಹಿತ", "ಪ್ರೀತಿ ಕಥೆ", "ಭಾವನೆ", "ನಟಸಾರ", "ಸಂದೇಶ", "ಆರಾಧನೆ", "ಚರಿತ್ರೆ", "ಸಂಬಂಧ", "ಪತ್ನಿ", "ಪತಿ", "ಪ್ರಿಯ", "ಪ್ರಣಯ", "ಪರಸ್ಪರ", "ಆಕರ್ಷಣ", "ವಲಯ", "ಪರಿಣಯ", "ಹೃದಯ", "ಹೃದಯಸ್ಪರ್ಶಿ", "ಚಿರಸ್ಮರಣೆ", "ಮಾಧುರ್ಯ", "ಅರ್ಥಪೂರ್ಣ", "ಮುದಿತ", "ಮಧುರತೆಯುಳ್ಳ", "ಪ್ರೀತಿಯೆಪ್ರೀತಿ", "ಅಡಿಗುಡಿಯಾಗು", "ಪರಿಪೂರ್ಣತೆ", "ಪವಿತ್ರತೆ", "ಹೃದಯಚಟುಪುಟಿ", "ಮಾಧುರ್ಯಭಾವನೆ", "ಆಧ್ಯಾತ್ಮಿಕ ಪ್ರೇಮ", "ಸ್ನೇಹಪೂರ್ಣ ಸಂಬಂಧ", "ಅಳಭಾವನೆ", "ಆತ್ಮೀಯತೆ", "ಅನಂತ ಪ್ರೇಮ", "ಮಧುರಸಂಭಾಷಣೆ", "ಆಸಕ್ತಿ"]
    }

    rasa_scores = {rasa: 0 for rasa in rasa_keywords}
    for rasa, keywords in rasa_keywords.items():
        for word in keywords:
            rasa_scores[rasa] += lyrics.count(word)

    max_rasa = max(rasa_scores, key=rasa_scores.get)
    return max_rasa if rasa_scores[max_rasa] > 0 else "Unknown"

# Function to scrape lyrics
def get_lyrics(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        lyrics_div = soup.find("div", class_="post-body")

        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
            return lyrics  # Return full lyrics instead of first 6-7 lines

        return "Lyrics not found"
    except Exception as e:
        return f"Error: {e}"

# Extract lyrics and classify
lyrics_data = []
for url in song_links:
    lyrics = get_lyrics(url)
    rasa = classify_rasa(lyrics)
    lyrics_data.append({"URL": url, "Lyrics": lyrics, "Rasa": rasa})

# Save to Excel
df = pd.DataFrame(lyrics_data)
df.to_excel("Kannada_Song_Classification.xlsx", index=False)
print("Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx")
from google.colab import files
files.download("Kannada_Song_Classification.xlsx")



Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

song_links =["https://sahityasudhe.blogspot.com/2012/01/blog-post_272.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2645.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_21.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BF.%E0%B2%B5%E0%B2%BF.%20%E0%B2%90%E0%B2%AF%E0%B3%8D%E0%B2%AF%E0%B2%B0%E0%B3%8D?updated-max=2013-01-10T21:43:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_46.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2462.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7423.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%81%E0%B2%9A%E0%B3%8D%E0%B2%9A%20%E0%B2%95%E0%B2%BF%E0%B2%9A%E0%B3%8D%E0%B2%9A?updated-max=2012-01-17T22:50:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B5%E0%B2%82%E0%B2%B6%E0%B2%BF?updated-max=2012-01-20T08:10:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8459.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AC%E0%B2%AF%E0%B2%B2%E0%B3%81%E0%B2%A6%E0%B2%BE%E0%B2%B0%E0%B2%BF?updated-max=2012-01-21T08:00:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1982_308.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B3%8D%E0%B2%B0%20%E0%B2%B6%E0%B3%87%E0%B2%96%E0%B2%B0%20%E0%B2%95%E0%B2%82%E0%B2%AC%E0%B2%BE%E0%B2%B0?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%A8%E0%B3%8B%E0%B2%AE%E0%B3%82%E0%B2%B0%E0%B3%8D%E0%B2%A4%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/1981_8109.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1809.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_937.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9344.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B5%E0%B3%8D%E0%B2%AF%E0%B2%BE%E0%B2%B8%E0%B2%B0%E0%B2%BE%E0%B2%AF%E0%B2%B0%E0%B3%81?updated-max=2017-09-06T10:06:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B3%81%E0%B2%B5%E0%B3%86%E0%B2%82%E0%B2%AA%E0%B3%81?updated-max=2013-01-18T21:30:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3664.html",
"https://sahityasudhe.blogspot.com/2013/01/1992.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B3%81?updated-max=2012-11-06T20:11:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2851.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6279.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AD%E0%B2%BE%E0%B2%97%E0%B3%8D%E0%B2%AF%E0%B2%A6%20%E0%B2%B2%E0%B2%95%E0%B3%8D%E0%B2%B7%E0%B3%8D%E0%B2%AE%E0%B3%80%20%E0%B2%AC%E0%B2%BE%E0%B2%B0%E0%B2%AE%E0%B3%8D%E0%B2%AE?updated-max=2012-01-23T19:19:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/7%27%E0%B2%93%20%E0%B2%95%E0%B3%8D%E0%B2%B2%E0%B2%BE%E0%B2%95%E0%B3%8D?updated-max=2012-04-25T23:11:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_27.html",
"https://sahityasudhe.blogspot.com/2012/06/2004_9119.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8520.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2187.html",
"https://sahityasudhe.blogspot.com/2019/09/1988.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_90.html",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_5807.html",
"https://sahityasudhe.blogspot.com/2012/01/1986.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3664.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_9.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1476.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B2%B5%E0%B2%B3%E0%B3%8D%E0%B2%B3%E0%B2%BF%E0%B2%AF%20%E0%B2%A4%E0%B3%8B%E0%B2%9F?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8540.html",
"https://sahityasudhe.blogspot.com/2012/11/blog-post_6.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2756.html",
"https://sahityasudhe.blogspot.com/2012/01/2010_4674.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7704.html",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_8430.html",
"https://sahityasudhe.blogspot.com/2019/09/ele-kenchi-taare-nammanetanka-baare.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_13.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B6%E0%B2%BE%E0%B2%82%E0%B2%A4%E0%B2%BF%20%E0%B2%95%E0%B3%8D%E0%B2%B0%E0%B2%BE%E0%B2%82%E0%B2%A4%E0%B2%BF?updated-max=2012-01-20T07:55:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%BF.%20%E0%B2%89%E0%B2%A6%E0%B2%AF%E0%B2%B6%E0%B2%82%E0%B2%95%E0%B2%B0%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_91.html",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_9991.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_218.html",
"https://sahityasudhe.blogspot.com/2013/01/1997_9.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%86%E0%B2%95%E0%B2%BE%E0%B2%B6%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6314.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%B0%E0%B2%BF%E0%B2%9A%E0%B2%AF?updated-max=2012-01-21T08:17:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2020/10/blog-post.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_84.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3965.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_20.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_46.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2014-02-04T23:17:00%2B05:30&max-results=20&start=122&by-date=false",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5341.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B3%81%E0%B2%B7%E0%B3%8D%E0%B2%AA%E0%B2%95%20%E0%B2%B5%E0%B2%BF%E0%B2%AE%E0%B2%BE%E0%B2%A8?updated-max=2017-08-31T12:30:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_84.html",
"https://sahityasudhe.blogspot.com/p/blog-page_22.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4815.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_13.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9090.html",
"https://sahityasudhe.blogspot.com/",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9344.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6826.html",
"https://sahityasudhe.blogspot.com/2012/10/blog-post_1398.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%8E%E0%B2%B0%E0%B2%A1%E0%B3%81%20%E0%B2%95%E0%B2%A8%E0%B2%B8%E0%B3%81?updated-max=2012-01-21T07:48:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6279.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4815.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%8F%E0%B2%B3%E0%B3%81%20%E0%B2%B8%E0%B3%81%E0%B2%A4%E0%B3%8D%E0%B2%A4%E0%B2%BF%E0%B2%A8%20%E0%B2%95%E0%B3%8B%E0%B2%9F%E0%B3%86?updated-max=2012-01-17T22:38:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/KGF?updated-max=2021-01-12T10:17:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2513.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6769.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1898.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2116.html",
"https://sahityasudhe.blogspot.com/2013/01/2002.html",
"https://sahityasudhe.blogspot.com/2012/11/blog-post_6.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_574.html"
"https://sahityasudhe.blogspot.com/2013/01/2002.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B0%E0%B2%BE%E0%B2%9C%20%E0%B2%A8%E0%B2%A8%E0%B3%8D%E0%B2%A8%20%E0%B2%B0%E0%B2%BE%E0%B2%9C?updated-max=2012-01-21T08:00:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/09/badavanadare-enu-priye.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2254.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3054.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%AF%20%E0%B2%AD%E0%B2%BE%E0%B2%B0%E0%B2%A4%20%E0%B2%9C%E0%B2%A8%E0%B2%A8%E0%B2%BF%E0%B2%AF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9923.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-18T21:31:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_83.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B3%88%E0%B2%B8%E0%B3%82%E0%B2%B0%E0%B3%81%20%E0%B2%AE%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%BF%E0%B2%97%E0%B3%86?updated-max=2012-10-19T23:19:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3640.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%BF%E0%B2%B2%E0%B2%A8?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/10/blog-post_9242.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_84.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5577.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2731.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A6%E0%B3%87%E0%B2%B5%E0%B2%A4%E0%B2%BE%20%E0%B2%AE%E0%B2%A8%E0%B3%81%E0%B2%B7%E0%B3%8D%E0%B2%AF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1879.html",
"https://sahityasudhe.blogspot.com/2017/09/satyavantara-sangaviralu.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_346.html",
"https://sahityasudhe.blogspot.com/2017/08/thirboki-jeevana-kirik-party-lyrics.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6826.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B2%BE%E0%B2%B9%E0%B2%B8%20%E0%B2%B8%E0%B2%BF%E0%B2%82%E0%B2%B9?updated-max=2012-01-23T19:18:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1982_24.html",
"https://sahityasudhe.blogspot.com/2012/10/blog-post_1398.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9900.html",
"https://sahityasudhe.blogspot.com/2012/06/2004.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3948.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3803.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_80.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-27T18:59:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9772.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_90.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_7802.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8496.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2513.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9900.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1898.html",
"https://sahityasudhe.blogspot.com/2012/06/1992_08.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_92.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3054.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_10.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9949.html",
"https://sahityasudhe.blogspot.com/2013/01/1997_9.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B3%86.%20%E0%B2%8E%E0%B2%B8%E0%B3%8D.%20%E0%B2%A8%E0%B2%BF%E0%B2%B8%E0%B3%8D%E0%B2%B8%E0%B2%BE%E0%B2%B0%E0%B3%8D%20%E0%B2%85%E0%B2%B9%E0%B3%8D%E0%B2%AE%E0%B3%86%E0%B2%A6%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3538.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_652.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-21T07:53:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2012/06/1996_08.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8939.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%BF.%E0%B2%B2%E0%B2%82%E0%B2%95%E0%B3%87%E0%B2%B6?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8448.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2734.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9399.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_45.html",
"https://sahityasudhe.blogspot.com/2012/01/1990_3432.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_49.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%92%E0%B2%B2%E0%B2%B5%E0%B2%BF%E0%B2%A8%20%E0%B2%89%E0%B2%A1%E0%B3%81%E0%B2%97%E0%B3%8A%E0%B2%B0%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4574.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3803.html",
"https://sahityasudhe.blogspot.com/2013/01/2011.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BF.%20%E0%B2%AA%E0%B2%BF.%20%E0%B2%B0%E0%B2%BE%E0%B2%9C%E0%B2%B0%E0%B2%A4%E0%B3%8D%E0%B2%A8%E0%B2%82?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6865.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_23.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9399.html",
"https://sahityasudhe.blogspot.com/2017/08/pushpaka-vimana-songs-lyrics.html",
"https://sahityasudhe.blogspot.com/2012/01/2009.html",
"https://sahityasudhe.blogspot.com/2012/11/1979.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B3%8A%E0%B2%A4%E0%B3%86%E0%B2%9C%E0%B3%8A%E0%B2%A4%E0%B3%86%E0%B2%AF%E0%B2%B2%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_34.html",
"https://sahityasudhe.blogspot.com/2020/08/title-track-of-mahabharata-serial.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1089.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_56.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3769.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-24T12:20:00%2B05:30&max-results=20&reverse-paginate=true&start=2&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/sad.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_49.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_80.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5434.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_9878.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B5%E0%B2%9A%E0%B2%A8%E0%B2%97%E0%B2%B3%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_442.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_90.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_520.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_7802.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_5530.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8147.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_652.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_10.html"
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8459.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3569.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_9.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_60.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_7098.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3275.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7423.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_909.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B3%81%E0%B2%AC%E0%B3%8D%E0%B2%AC%E0%B2%B0%E0%B2%BE%E0%B2%AF%20%E0%B2%9A%E0%B2%95%E0%B3%8D%E0%B2%95%E0%B3%8A%E0%B2%A1%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/11/blog-post_3918.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1002.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3569.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6378.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1809.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_3809.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_1840.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-10T21:57:00%2B05:30&max-results=20&start=222&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1981_8109.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5341.html",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_3072.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_27.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_49.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4086.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2332.html",
"https://sahityasudhe.blogspot.com/2012/11/blog-post.html"
]

def classify_rasa(lyrics):
    rasa_keywords = {
        "Devotional": ["ದೇವ", "ಭಕ್ತಿ", "ಪೂಜೆ", "ಹರಿಯೆ", "ಶರಣು", "ರಾಮ", "ಕೃಷ್ಣ", "ವಿಷ್ಣು", "ಪರಮಾತ್ಮ", "ಆರಾಧನೆ", "ಧರ್ಮ", "ಪರಮೇಶ್ವರ", "ಭಕ್ತರು", "ಅರ್ಚನೆ", "ಗೋವಿಂದ", "ಶಿವ", "ದೇವತೆ", "ಅಷ್ಟೋತ್ತರ", "ವಂದನೆ", "ಪೂಜಾರಿ", "ಪವಿತ್ರ", "ಸಂಸ್ಕಾರ", "ಆರಾಧನೆ", "ಸಾಧನೆ", "ಯೋಗ", "ಧ್ಯಾನ", "ಹನುಮಾನ್", "ಪರಮಭಕ್ತ", "ಗುರು", "ದೀಕ್ಷೆ", "ಪ್ರಾರ್ಥನೆ", "ಸಾನ್ನಿಧ್ಯ", "ಶಕ್ತಿ", "ಅಧ್ಯಾತ್ಮ", "ಆತ್ಮದರ್ಶನ", "ಪೂಜಿತ", "ಸಾಗರ", "ಪಾರಮಾರ್ಥ", "ಅವತಾರ", "ಪುನರ್ಜನ್ಮ", "ಬ್ರಹ್ಮ", "ಮೋಕ್ಷ", "ಪುನೀತ", "ಜಪ", "ಸ್ತೋತ್ರ", "ಅಂತ್ಯಕಥೆ", "ಆಧ್ಯಾತ್ಮಿಕ", "ಅಮೃತ", "ಚಕ್ರ"],
        "Patriotic": ["ದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ", "ಜೈಹಿಂದ್", "ಸಮರ್ಪಣೆ", "ಭಾರತ", "ಧ್ವಜ", "ಸ್ವಾಭಿಮಾನ", "ವೀರ", "ರಾಜ್ಯ", "ಏಕತೆ", "ರಾಷ್ಟ್ರ", "ಬಾಳಿದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ ಹೋರಾಟ", "ಭಾರತ ಮಾತಾ", "ಜೈ ಭಾರತ", "ಭಾರತೀಯ", "ಹೋರಾಟಗಾರ", "ಬಸವಣ್ಣ", "ಸ್ಪರ್ಧೆ", "ವೀರಸೇನೆ", "ಸಮಾನತೆ", "ಪ್ರಜಾಸತ್ತಾತ್ಮಕ", "ಜಾತಿ", "ರಾಷ್ಟ್ರಪ್ರೇಮ", "ಸೈನಿಕ", "ಸಾಮರಸ್ಯ", "ಹಕ್ಕು", "ವಿದ್ಯಾರ್ಥಿ", "ಸಂಘಟನೆ", "ಸಂಭ್ರಮ", "ಬಿಗ್ ಡೇ", "ಅಜಯ್ ಭಾರತ", "ದೇಶಭಕ್ತ", "ಪಾಲ್ಗೊಂಡರು", "ಹೋರಾಟ", "ಆಂದೋಲನ", "ಸಂಬಂಧ", "ಹೈದರಾಬಾದ್", "ಗೌರವ", "ಆತ್ಮವಿಶ್ವಾಸ", "ಧೈರ್ಯ", "ಅಖಂಡ ಭಾರತ", "ಪರಾಕ್ರಮ", "ರಕ್ಷಕ", "ಪರಮವೀರ", "ಪರಿಪೂರ್ಣ", "ಶೌರ್ಯ", "ವಿಜಯ", "ವಿಜಯೋತ್ಸವ"],
        "Folk": ["ಜನಪದ", "ಹೊಲವು", "ಕುಟುಕು", "ಗ್ರಾಮೀಣ", "ಬಾಲಪಾಟ", "ಕುಣಿತ", "ನಾಡು", "ಹಬ್ಬ", "ಹಳ್ಳಿ", "ಕೃತನೆ", "ಹೆಜ್ಜೆ", "ಹೊರಾಟ", "ಬಾವಲಿ", "ದೋಣಿ", "ಮುದ್ದು", "ಬಡವರ", "ಬೂದಿ", "ಮಳಗು", "ಪಾದ", "ಬದನೆ", "ಹಳ್ಳಿಗುಡಿ", "ಕಾಯ್ಕಳು", "ಕಬ್ಬಿಣ", "ಕೃಷಿ", "ಬಟ್ಟಲು", "ಹಾದಿ", "ಬೀದಿ", "ಅಂಗಡಿ", "ರಸಗೆಜೆ", "ಗೊಡ್ಡಲು", "ಅರವಟ್ಟೆ", "ಅಕ್ಕಸಾಲಿಗೆ", "ಆಡುಗೊಳಲು", "ಅಕ್ಕಪಕ್ಕ", "ಹೂವಿನ ಹಾರ", "ಮಡಕೆ", "ಮೂಡಲಗೇರಿ", "ಮಕ್ಕಳಾಟ", "ನೆಲದಾಟ", "ಮಾಲಿಗೆ", "ಚಪ್ಪಲಿ", "ಬಸವಣ", "ದೀಪಾಲಂಕಾರ", "ಗಂಗಾವತಿ", "ಕಂಭಳ", "ಹೆಜ್ಜೆಗುಂಡಿ", "ಮರಹರಕೆ", "ಚಂದ್ರಹಾರ", "ಪಡುಗೆಯಾಟ", "ಬೂತಕೋಲ"],
        "Happy": ["ನಗುವು", "ಸಂತೋಷ", "ಹರ್ಷ", "ಆನಂದ", "ಉಲ್ಲಾಸ", "ಮೋಹಕ", "ಸಂತೃಪ್ತಿ", "ಹರ್ಷೋಲ್ಲಾಸ", "ಉತ್ಸವ", "ಸುವಾಸನೆ", "ಶ್ರದ್ಧೆ", "ಪ್ರಸನ್ನತೆ", "ಪ್ರಕೃತಿ", "ಶಕ್ತಿಮಂತ್ರ", "ಆಲಾಪನೆ", "ಆಹ್ಲಾದ", "ಚಿರಂಜೀವಿ", "ಜೋಶ್", "ಸ್ಫೂರ್ತಿ", "ಮೋಜು", "ಹಾಸ್ಯ", "ಪ್ರಜ್ಞೆ", "ಸಂಬ್ರಮ", "ಆಕರ್ಷಣ", "ಮಹೋತ್ಸವ", "ಮಹಿಮಾ", "ಪ್ರಾರ್ಥನೆ", "ಆನಂದಿತ", "ಮೋಜುಗಾಡು", "ಚಲನೆ", "ಮೋಹನೆ", "ಅಮೃತ", "ಆನಂದಧಾರೆ", "ಹಬ್ಬದಚಟುವಟಿಕೆಗಳು", "ಸಂಗೀತ", "ಜಶ್ನ್", "ಅದ್ಭುತ", "ವಿಶೇಷ", "ಹೃತ್ಪೂರ್ವಕ", "ಆರಂಭ", "ಆನಂದೋತ್ಸವ", "ಪ್ರಸನ್ನಮುಖ", "ಸುಖಪೂರ್ಣ", "ಹಾಸ್ಯಚಟುವಟಿಕೆ", "ಸ್ನೇಹಪೂರ್ಣ", "ಚಿರಪರಿಚಿತ", "ಪ್ರೋತ್ಸಾಹ", "ಸಿರಿವಂತಿಕೆ", "ಸುದೀರ್ಘ"],
        "Sad": ["ದುಃಖ", "ಕಣ್ಣೀರು", "ನೋವು", "ಬಾಳು", "ವಿಚಾರ", "ಕಳವಳ", "ಮರುಳು", "ಪೀಡೆ", "ಆರತಿ", "ವಿಪತ್ತು", "ತೀವ್ರ ನೋವು", "ಬಿರುಗಾಳಿ", "ಮರಣ", "ಆಲೋಚನೆ", "ಪರಿಶುದ್ಧತೆ", "ಬೇಸರ", "ಪಾತಕ", "ನಷ್ಟ", "ವಿರಹ", "ಆರಾಧನೆ", "ಆಶ್ರು", "ಬದ್ಧ", "ಮೋಹಪಾಶ", "ಅನಾಥ", "ವಿದಾಯ", "ಆಳ್ವಿಕೆ", "ಅಪಘಾತ", "ಶೋಕ", "ಅಪರಾಧ ಭಾವನೆ", "ಪತಿತ", "ಪತನ", "ಹೃದಯವೇದನೆ", "ವಿಚಲನೆ", "ಆರಂಬವೇದನೆ", "ಅಪಮಾನ", "ಅಸಹಾಯಕತೆ", "ಬಿರುಗಾಳಿ", "ಅಪಶ್ರಯ", "ಹೀನತೆಗೆ", "ದ್ವೇಷ", "ಅಧ್ಯಾತ್ಮ ಪೀಡೆ", "ಮರಮಾತು", "ಅಪಸ್ಮೃತಿ", "ಅನಿರೀಕ್ಷಿತ ಪೀಡೆ", "ಹೀನಗತಿ", "ಮೂಢತನ", "ಅಪಮಾನಾತ್ಮಕ", "ಬಿದಿರಹಿತ", "ಹೀನಭಾವ"],
        "Fear": ["ಭಯ", "ಗಾಬರಿ", "ಅಜ್ಞಾನ", "ವಿಕಂಪ", "ಭೀತಿ", "ಆತಂಕ", "ಹೆದರಿಕೆ", "ಭ್ರಾಂತಿ", "ಕಂಪನ", "ಅಪಾಯ", "ಆಘಾತ", "ಅಪಘಾತ", "ಅಪರೂಪ", "ಹಿಡಿದುಕೊಳ್ಳು", "ಅಪರಿಚಿತ", "ಪೀಡೆ", "ಅಪರಾಧ", "ಆಸ್ಪದ", "ಬಿಥ್", "ಅಪಾಯಕಾರಿ", "ಆಕಸ್ಮಿಕ", "ಅಪಾಯಕಾರಕ", "ಅಪಾಯಕಾರಿ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ", "ಅಪಹರಣ", "ಅಪಹಾಸ್ಯ", "ಅನಿಶ್ಚಿತತೆ", "ಅಪಸ್ವರ", "ಅಪಪ್ರಚಾರ", "ಅಪಾಧರಿತ", "ಅಪರಿಚಿತ", "ಅಪಾಯಸೂಚಕ", "ಹಿಂಸೆ", "ಅಪಲೋಚನೆ", "ಅಪರೂಪವಾದ", "ಅಪರಾಧಿ", "ಅಪಮಾನದಾಯಕ", "ಅಪಾಯಕಾರಿ ದಾರಿ", "ಅಪಾಯದ ಕ್ಷಣ", "ಭೀತಿಗೊಳಗಾಗು", "ಆತಂಕಭರಿತ", "ಆಘಾತ ಭರಿತ", "ಅಪಮಾನಿತ", "ಭಯಾನಕ ದೃಶ್ಯ", "ಅಪಯಶಸ್ವಿ", "ಅಪಾಯಕಾರಕ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ ಘಟನೆ", "ಅಪಹೃತ"],
        "Love": ["ಪ್ರೇಮ", "ಮನಸೆ", "ನೀನು", "ನನ್ನ", "ಪ್ರೀತಿ", "ಸ್ನೇಹ", "ಅನುರಾಗ", "ಆಕರ್ಷಣೆ", "ಮಧುರ", "ಅಭಿಮಾನ", "ರೋಮ್ಯಾಂಸ್", "ವೈಭವ", "ಸ್ನೇಹಭಾವ", "ಮೋಹಿತ", "ಪ್ರೀತಿ ಕಥೆ", "ಭಾವನೆ", "ನಟಸಾರ", "ಸಂದೇಶ", "ಆರಾಧನೆ", "ಚರಿತ್ರೆ", "ಸಂಬಂಧ", "ಪತ್ನಿ", "ಪತಿ", "ಪ್ರಿಯ", "ಪ್ರಣಯ", "ಪರಸ್ಪರ", "ಆಕರ್ಷಣ", "ವಲಯ", "ಪರಿಣಯ", "ಹೃದಯ", "ಹೃದಯಸ್ಪರ್ಶಿ", "ಚಿರಸ್ಮರಣೆ", "ಮಾಧುರ್ಯ", "ಅರ್ಥಪೂರ್ಣ", "ಮುದಿತ", "ಮಧುರತೆಯುಳ್ಳ", "ಪ್ರೀತಿಯೆಪ್ರೀತಿ", "ಅಡಿಗುಡಿಯಾಗು", "ಪರಿಪೂರ್ಣತೆ", "ಪವಿತ್ರತೆ", "ಹೃದಯಚಟುಪುಟಿ", "ಮಾಧುರ್ಯಭಾವನೆ", "ಆಧ್ಯಾತ್ಮಿಕ ಪ್ರೇಮ", "ಸ್ನೇಹಪೂರ್ಣ ಸಂಬಂಧ", "ಅಳಭಾವನೆ", "ಆತ್ಮೀಯತೆ", "ಅನಂತ ಪ್ರೇಮ", "ಮಧುರಸಂಭಾಷಣೆ", "ಆಸಕ್ತಿ"]
    }

    rasa_scores = {rasa: 0 for rasa in rasa_keywords}
    for rasa, keywords in rasa_keywords.items():
        for word in keywords:
            rasa_scores[rasa] += lyrics.count(word)

    max_rasa = max(rasa_scores, key=rasa_scores.get)
    return max_rasa if rasa_scores[max_rasa] > 0 else "Unknown"

# Function to scrape lyrics
def get_lyrics(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        lyrics_div = soup.find("div", class_="post-body")

        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
            return lyrics  # Return full lyrics instead of first 6-7 lines

        return "Lyrics not found"
    except Exception as e:
        return f"Error: {e}"

# Extract lyrics and classify
lyrics_data = []
for url in song_links:
    lyrics = get_lyrics(url)
    rasa = classify_rasa(lyrics)
    lyrics_data.append({"URL": url, "Lyrics": lyrics, "Rasa": rasa})

# Save to Excel
df = pd.DataFrame(lyrics_data)
df.to_excel("Kannada_Song_Classification.xlsx", index=False)
print("Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx")
from google.colab import files
files.download("Kannada_Song_Classification.xlsx")



Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of song URLs
song_links = [
"https://sahityasudhe.blogspot.com/2013/01/2006_2716.html",
"https://sahityasudhe.blogspot.com/2012/06/2004.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_593.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6314.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B3%80%E0%B2%A8%E0%B3%86%20%E0%B2%AC%E0%B2%B0%E0%B2%BF%20%E0%B2%A8%E0%B3%80%E0%B2%A8%E0%B3%86?updated-max=2012-04-25T23:18:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8459.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%B0%E0%B2%AE%E0%B2%A8%E0%B3%86?updated-max=2012-01-20T07:40:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9828.html",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu_9.html",
"https://sahityasudhe.blogspot.com/2012/06/1996.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_25.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3275.html",
"https://sahityasudhe.blogspot.com/2012/04/1984.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_909.html",
"https://sahityasudhe.blogspot.com/2012/06/2006_6008.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-17T22:38:00%2B05:30&max-results=20&start=542&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3664.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7448.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%82%E0%B2%AC%E0%B2%BE%E0%B2%B0%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5239.html",
"https://sahityasudhe.blogspot.com/2014/03/blog-post.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%87%E0%B2%AC%E0%B3%8D%E0%B2%AC%E0%B2%A8%E0%B2%BF%20%E0%B2%95%E0%B2%B0%E0%B2%97%E0%B2%BF%E0%B2%A4%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/2020/10/blog-post.html",
"https://sahityasudhe.blogspot.com/2012/06/2006_6668.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5218.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1525.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%86%E0%B2%B0%E0%B3%8D.%E0%B2%8E%E0%B2%A8%E0%B3%8D.%E0%B2%9C%E0%B2%AF%E0%B2%97%E0%B3%8B%E0%B2%AA%E0%B2%BE%E0%B2%B2%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2851.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7605.html",
"https://sahityasudhe.blogspot.com/2013/01/2006_9.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_72.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4919.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_27.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5237.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6842.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B3%88%E0%B2%95%E0%B3%8B?updated-max=2012-01-20T08:03:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9828.html",
"https://sahityasudhe.blogspot.com/2012/01/1983.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_44.html",
"https://sahityasudhe.blogspot.com/2012/06/1996_7491.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9198.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_13.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9934.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5536.html",
"https://sahityasudhe.blogspot.com/2019/07/sad.html",
"https://sahityasudhe.blogspot.com/2012/04/1984_25.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5345.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8496.html",
"https://sahityasudhe.blogspot.com/2012/06/1997_08.html",
"https://sahityasudhe.blogspot.com/2017/09/yenda-hendthi-kannada-padagol-g-p.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-23T19:20:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2226.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_518.html",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_8879.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2981.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7619.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%A8%E0%B3%8D%E0%B2%AE%20%E0%B2%9C%E0%B2%A8%E0%B3%8D%E0%B2%AE%E0%B2%A6%20%E0%B2%85%E0%B2%A8%E0%B3%81%E0%B2%AC%E0%B2%82%E0%B2%A7?updated-max=2012-01-20T08:07:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1525.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8098.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AC%E0%B3%86%E0%B2%82%E0%B2%95%E0%B2%BF%E0%B2%AF%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%BF%20%E0%B2%85%E0%B2%B0%E0%B2%B3%E0%B2%BF%E0%B2%A6%20%E0%B2%B9%E0%B3%82%E0%B2%B5%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%B5%E0%B2%BF%20%E0%B2%85%E0%B2%A8%E0%B3%81%E0%B2%AA%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%B5%E0%B2%BF?updated-max=2012-01-21T08:03:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7704.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3948.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6826.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9772.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BE%E0%B2%A8%E0%B2%BF%20%E0%B2%AE%E0%B3%87%E0%B2%B0%E0%B2%BE%20%E0%B2%A8%E0%B2%BE%E0%B2%AE%E0%B3%8D%20%E0%B2%AA%E0%B3%8D%E0%B2%B0%E0%B3%80%E0%B2%A4%E0%B2%BF%20%E0%B2%AE%E0%B3%87%E0%B2%B0%E0%B2%BE?&max-results=5",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_72.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3925.html",
"https://sahityasudhe.blogspot.com/2019/09/innu-yaka-baralillavva-hubballiyava.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8098.html",
"https://sahityasudhe.blogspot.com/2017/09/pillangoviya-cheluva.html",
"https://sahityasudhe.blogspot.com/2019/09/1988.html",
"https://sahityasudhe.blogspot.com/2012/04/1984.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7704.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9524.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4099.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2693.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1084.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_9459.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3538.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%B0%E0%B2%B3%E0%B2%BF%20%E0%B2%B9%E0%B3%82%E0%B2%B5%E0%B2%BE%E0%B2%97%E0%B2%BF?updated-max=2012-06-12T22:32:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2851.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9456.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7294.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_6863.html",
"https://sahityasudhe.blogspot.com/2017/08/last-bench-party-kirik-party-lyrics.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_28.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_42.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2731.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_15.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3769.html",
"https://sahityasudhe.blogspot.com/2012/06/2006.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B3%86%E0%B2%A8%E0%B2%AA%E0%B2%BF%E0%B2%B0%E0%B2%B2%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6047.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5144.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9934.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5770.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_20.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9923.html",
"https://sahityasudhe.blogspot.com/2012/01/1982_5778.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5345.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_8768.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BF.%E0%B2%B5%E0%B2%BF.%20%E0%B2%90%E0%B2%AF%E0%B3%8D%E0%B2%AF%E0%B2%B0%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/09/yenda-hendthi-kannada-padagol-g-p.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%93%E0%B2%82?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8520.html",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_8879.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%B3%E0%B3%86%E0%B2%AF%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%BF%20%E0%B2%9C%E0%B3%8A%E0%B2%A4%E0%B3%86%E0%B2%AF%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%BF?updated-max=2012-01-21T08:17:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/09/huttida-oorannu-bittu-banda-mele.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_25.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-06-12T22:32:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7619.html"
"https://sahityasudhe.blogspot.com/2019/09/ele-kenchi-taare-nammanetanka-baare.html"
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%97%E0%B2%BE%E0%B2%A8%20%E0%B2%AC%E0%B2%9C%E0%B2%BE%E0%B2%A8?updated-max=2012-01-23T19:24:00%2B05:30&max-results=5&start=5&by-date=false"
"https://sahityasudhe.blogspot.com/2017/08/katheyonda-helide-kirik-party-songs.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_97.html"
"https://sahityasudhe.blogspot.com/2017/08/blog-post_24.html"
"https://sahityasudhe.blogspot.com/2012/04/2009_8768.html"
"https://sahityasudhe.blogspot.com/2013/01/blog-post_444.html"
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8141.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3948.html",
"https://sahityasudhe.blogspot.com/2019/09/innu-yaka-baralillavva-hubballiyava.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_120.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_96.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_72.html",
"https://sahityasudhe.blogspot.com/2013/01/1985.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_9995.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%97%E0%B3%8B%E0%B2%95%E0%B3%81%E0%B2%B2?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9772.html",
"https://sahityasudhe.blogspot.com/2017/09/blog-post.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9817.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B3%80%E0%B2%B5%E0%B2%A8%20%E0%B2%9A%E0%B3%88%E0%B2%A4%E0%B3%8D%E0%B2%B0?updated-max=2012-06-08T22:34:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9567.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5348.html",
"https://sahityasudhe.blogspot.com/2013/01/2006_2716.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9934.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2693.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B3%80%E0%B2%B5%E0%B2%A8%20%E0%B2%9A%E0%B2%95%E0%B3%8D%E0%B2%B0?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5239.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3054.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4086.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9093.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6521.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1876.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3266.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5682.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AC%E0%B2%AF%E0%B2%B2%E0%B3%81%E0%B2%A6%E0%B2%BE%E0%B2%B0%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6047.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6992.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B6%E0%B3%83%E0%B2%82%E0%B2%97%E0%B2%BE%E0%B2%B0%20%E0%B2%95%E0%B2%BE%E0%B2%B5%E0%B3%8D%E0%B2%AF?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_22.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B2%BE%E0%B2%82%E0%B2%A6%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9923.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9631.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_20.html",
"https://sahityasudhe.blogspot.com/2012/06/2004_960.html",
"https://sahityasudhe.blogspot.com/2017/08/kirik-party-kagadada-doniyalli-lyrics.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3782.html",
"https://sahityasudhe.blogspot.com/2012/01/2010.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_60.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5144.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4350.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2462.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_444.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5439.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4973.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5816.html",
"https://sahityasudhe.blogspot.com/2017/09/baarisu-kannada-dindimava.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B2%BE%E0%B2%A1%E0%B2%97%E0%B3%80%E0%B2%A4%E0%B3%86?updated-max=2013-01-16T08:01:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_47.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-20T07:53:00%2B05:30&max-results=20&start=502&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/blog-post.html",
"https://sahityasudhe.blogspot.com/2012/01/1983_17.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_25.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_91.html",
"https://sahityasudhe.blogspot.com/2012/11/2012.html",
"https://sahityasudhe.blogspot.com/2019/09/ele-kenchi-taare-nammanetanka-baare.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5241.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%B5%E0%B2%BF%E0%B2%B0%E0%B2%A4%E0%B3%8D%E0%B2%A8%20%E0%B2%95%E0%B2%BE%E0%B2%B3%E0%B2%BF%E0%B2%A6%E0%B2%BE%E0%B2%B8?updated-max=2012-01-21T08:03:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_39.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3266.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1957.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_4459.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7610.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3782.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5395.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7728.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_9158.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B2%BE%E0%B2%82%E0%B2%A6%E0%B2%BF?updated-max=2012-01-21T07:49:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/1992.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5434.html",
"https://sahityasudhe.blogspot.com/2013/01/2006_2716.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-04-25T23:15:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_4.html",
"https://sahityasudhe.blogspot.com/2019/09/badavanadare-enu-priye.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7132.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_67.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3053.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9567.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5085.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1083.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8007.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3675.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A6%E0%B2%BE%E0%B2%B0%E0%B2%BF%20%E0%B2%A4%E0%B2%AA%E0%B3%8D%E0%B2%AA%E0%B2%BF%E0%B2%A6%20%E0%B2%AE%E0%B2%97?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%AD%E0%B2%AF%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2402.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3275.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4099.html",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_25.html",
"https://sahityasudhe.blogspot.com/2012/01/2010.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_13.html",
"https://sahityasudhe.blogspot.com/2012/01/1965.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%97%E0%B2%82%E0%B2%A7%E0%B2%A6%20%E0%B2%97%E0%B3%81%E0%B2%A1%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B0%E0%B2%BE%E0%B2%9C%E0%B3%8D%20-%20The%20showman?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/2002_3660.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_45.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4604.html",
"https://sahityasudhe.blogspot.com/2013/01/2006.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_10.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_28.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-28T12:05:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7931.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2975.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_7098.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5922.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_8648.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%AA%E0%B3%8D%E0%B2%AA%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%BF%E0%B2%A4%E0%B3%8D%E0%B2%B0%E0%B2%97%E0%B3%80%E0%B2%A4%E0%B3%86%E0%B2%97%E0%B2%B3%E0%B3%81?updated-max=2019-07-13T15:25:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1981_1258.html",
"https://sahityasudhe.blogspot.com/2013/01/1987.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1957.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-28T12:08:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AF%E0%B3%81%E0%B2%97%E0%B2%AA%E0%B3%81%E0%B2%B0%E0%B3%81%E0%B2%B7?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4773.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_92.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6314.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B2%E0%B2%B5%E0%B3%8D%20%E0%B2%97%E0%B3%81%E0%B2%B0%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%93%20%E0%B2%AE%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%BF%E0%B2%97%E0%B3%86?updated-max=2012-04-25T08:21:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B2%E0%B3%88%E0%B2%AB%E0%B3%81%20%E0%B2%87%E0%B2%B7%E0%B3%8D%E0%B2%9F%E0%B3%87%E0%B2%A8%E0%B3%87?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/1981_6449.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_272.html",
"https://sahityasudhe.blogspot.com/2019/09/badavanadare-enu-priye.html",
"https://sahityasudhe.blogspot.com/2012/01/1992.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_24.html",
"https://sahityasudhe.blogspot.com/2013/01/1985.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9631.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1166.html",
"https://sahityasudhe.blogspot.com/2013/01/1982_9.html",
"https://sahityasudhe.blogspot.com/2012/11/2012_6.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7361.html",
"https://sahityasudhe.blogspot.com/2013/01/2011.html",
"https://sahityasudhe.blogspot.com/2012/01/2010_531.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_721.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-09-05T14:45:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%82%E0%B2%9A%E0%B2%B0%E0%B2%82%E0%B2%97%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_48.html"
"https://sahityasudhe.blogspot.com/2012/06/blog-post.html"
"https://sahityasudhe.blogspot.com/2017/08/blog-post_97.html"
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-25T08:00:00%2B05:30&max-results=20&reverse-paginate=true"
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B3%8A%E0%B2%B8%E0%B3%86%20%E0%B2%A4%E0%B2%82%E0%B2%A6%20%E0%B2%B8%E0%B3%8C%E0%B2%AD%E0%B2%BE%E0%B2%97%E0%B3%8D%E0%B2%AF?updated-max=2012-01-20T08:07:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_45.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_97.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_27.html",
"https://sahityasudhe.blogspot.com/2013/01/2009.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9456.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2132.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_25.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7931.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4773.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3811.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B2%BE%E0%B2%B9%E0%B2%B8%20%E0%B2%B8%E0%B2%BF%E0%B2%82%E0%B2%B9?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5237.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_49.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_8648.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6842.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2116.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_1258.html",
"https://sahityasudhe.blogspot.com/2013/01/1987.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_97.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2369.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B2%BF%E0%B2%A6%E0%B3%8D%E0%B2%A7%E0%B2%B2%E0%B2%BF%E0%B2%82%E0%B2%97%E0%B2%AF%E0%B3%8D%E0%B2%AF?updated-max=2012-01-17T22:43:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4084.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_20.html",
"https://sahityasudhe.blogspot.com/2017/09/bhagyada-lakshmi-baramma.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_8093.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5488.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_346.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7361.html",
"https://sahityasudhe.blogspot.com/2012/01/1992_17.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-28T12:00:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3047.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_8093.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9198.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%BF%E0%B2%B2%E0%B2%A8?updated-max=2012-01-20T07:54:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2466.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9631.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7931.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_604.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_195.html",
"https://sahityasudhe.blogspot.com/2013/01/2011.html",
"https://sahityasudhe.blogspot.com/2012/01/2010_531.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2015.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8141.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-09T21:26:00%2B05:30&max-results=20&start=282&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4464.html",
"https://sahityasudhe.blogspot.com/2012/06/blog-post.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_36.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%97%E0%B2%82%E0%B2%A7%E0%B2%A6%20%E0%B2%97%E0%B3%81%E0%B2%A1%E0%B2%BF?updated-max=2012-01-21T07:56:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1983.html",
"https://sahityasudhe.blogspot.com/2012/01/paramatma.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2423.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2975.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%83%E0%B2%A6%E0%B2%AF%20%E0%B2%AA%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%B5%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2014/02/blog-post.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7473.html",
"https://sahityasudhe.blogspot.com/",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%A7%E0%B3%81%E0%B2%B0%20%E0%B2%B8%E0%B2%82%E0%B2%97%E0%B2%AE?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6546.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1932.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_49.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7605.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A6%E0%B3%81%E0%B2%A8%E0%B2%BF%E0%B2%AF%E0%B2%BE?updated-max=2012-01-20T08:08:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/09/ivale-veena-paani.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2019-07-13T15:15:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2280.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2951.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_39.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_18.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_8093.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4263.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_24.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2015.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5144.html",
"https://sahityasudhe.blogspot.com/2012/10/blog-post_9242.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_325.html",
"https://sahityasudhe.blogspot.com/2012/01/1965.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_0.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B3%8D%E0%B2%B0%E0%B3%87%E0%B2%AE%E0%B2%A6%20%E0%B2%95%E0%B2%BE%E0%B2%A3%E0%B2%BF%E0%B2%95%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%A8%E0%B3%81%E0%B2%B0%E0%B2%BE%E0%B2%97%20%E0%B2%85%E0%B2%B0%E0%B2%B3%E0%B2%BF%E0%B2%A4%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/09/bhagyada-lakshmi-baramma.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9198.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6583.html"

]

def classify_rasa(lyrics):
    rasa_keywords = {
        "Devotional": ["ದೇವ", "ಭಕ್ತಿ", "ಪೂಜೆ", "ಹರಿಯೆ", "ಶರಣು", "ರಾಮ", "ಕೃಷ್ಣ", "ವಿಷ್ಣು", "ಪರಮಾತ್ಮ", "ಆರಾಧನೆ", "ಧರ್ಮ", "ಪರಮೇಶ್ವರ", "ಭಕ್ತರು", "ಅರ್ಚನೆ", "ಗೋವಿಂದ", "ಶಿವ", "ದೇವತೆ", "ಅಷ್ಟೋತ್ತರ", "ವಂದನೆ", "ಪೂಜಾರಿ", "ಪವಿತ್ರ", "ಸಂಸ್ಕಾರ", "ಆರಾಧನೆ", "ಸಾಧನೆ", "ಯೋಗ", "ಧ್ಯಾನ", "ಹನುಮಾನ್", "ಪರಮಭಕ್ತ", "ಗುರು", "ದೀಕ್ಷೆ", "ಪ್ರಾರ್ಥನೆ", "ಸಾನ್ನಿಧ್ಯ", "ಶಕ್ತಿ", "ಅಧ್ಯಾತ್ಮ", "ಆತ್ಮದರ್ಶನ", "ಪೂಜಿತ", "ಸಾಗರ", "ಪಾರಮಾರ್ಥ", "ಅವತಾರ", "ಪುನರ್ಜನ್ಮ", "ಬ್ರಹ್ಮ", "ಮೋಕ್ಷ", "ಪುನೀತ", "ಜಪ", "ಸ್ತೋತ್ರ", "ಅಂತ್ಯಕಥೆ", "ಆಧ್ಯಾತ್ಮಿಕ", "ಅಮೃತ", "ಚಕ್ರ"],
        "Patriotic": ["ದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ", "ಜೈಹಿಂದ್", "ಸಮರ್ಪಣೆ", "ಭಾರತ", "ಧ್ವಜ", "ಸ್ವಾಭಿಮಾನ", "ವೀರ", "ರಾಜ್ಯ", "ಏಕತೆ", "ರಾಷ್ಟ್ರ", "ಬಾಳಿದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ ಹೋರಾಟ", "ಭಾರತ ಮಾತಾ", "ಜೈ ಭಾರತ", "ಭಾರತೀಯ", "ಹೋರಾಟಗಾರ", "ಬಸವಣ್ಣ", "ಸ್ಪರ್ಧೆ", "ವೀರಸೇನೆ", "ಸಮಾನತೆ", "ಪ್ರಜಾಸತ್ತಾತ್ಮಕ", "ಜಾತಿ", "ರಾಷ್ಟ್ರಪ್ರೇಮ", "ಸೈನಿಕ", "ಸಾಮರಸ್ಯ", "ಹಕ್ಕು", "ವಿದ್ಯಾರ್ಥಿ", "ಸಂಘಟನೆ", "ಸಂಭ್ರಮ", "ಬಿಗ್ ಡೇ", "ಅಜಯ್ ಭಾರತ", "ದೇಶಭಕ್ತ", "ಪಾಲ್ಗೊಂಡರು", "ಹೋರಾಟ", "ಆಂದೋಲನ", "ಸಂಬಂಧ", "ಹೈದರಾಬಾದ್", "ಗೌರವ", "ಆತ್ಮವಿಶ್ವಾಸ", "ಧೈರ್ಯ", "ಅಖಂಡ ಭಾರತ", "ಪರಾಕ್ರಮ", "ರಕ್ಷಕ", "ಪರಮವೀರ", "ಪರಿಪೂರ್ಣ", "ಶೌರ್ಯ", "ವಿಜಯ", "ವಿಜಯೋತ್ಸವ"],
        "Folk": ["ಜನಪದ", "ಹೊಲವು", "ಕುಟುಕು", "ಗ್ರಾಮೀಣ", "ಬಾಲಪಾಟ", "ಕುಣಿತ", "ನಾಡು", "ಹಬ್ಬ", "ಹಳ್ಳಿ", "ಕೃತನೆ", "ಹೆಜ್ಜೆ", "ಹೊರಾಟ", "ಬಾವಲಿ", "ದೋಣಿ", "ಮುದ್ದು", "ಬಡವರ", "ಬೂದಿ", "ಮಳಗು", "ಪಾದ", "ಬದನೆ", "ಹಳ್ಳಿಗುಡಿ", "ಕಾಯ್ಕಳು", "ಕಬ್ಬಿಣ", "ಕೃಷಿ", "ಬಟ್ಟಲು", "ಹಾದಿ", "ಬೀದಿ", "ಅಂಗಡಿ", "ರಸಗೆಜೆ", "ಗೊಡ್ಡಲು", "ಅರವಟ್ಟೆ", "ಅಕ್ಕಸಾಲಿಗೆ", "ಆಡುಗೊಳಲು", "ಅಕ್ಕಪಕ್ಕ", "ಹೂವಿನ ಹಾರ", "ಮಡಕೆ", "ಮೂಡಲಗೇರಿ", "ಮಕ್ಕಳಾಟ", "ನೆಲದಾಟ", "ಮಾಲಿಗೆ", "ಚಪ್ಪಲಿ", "ಬಸವಣ", "ದೀಪಾಲಂಕಾರ", "ಗಂಗಾವತಿ", "ಕಂಭಳ", "ಹೆಜ್ಜೆಗುಂಡಿ", "ಮರಹರಕೆ", "ಚಂದ್ರಹಾರ", "ಪಡುಗೆಯಾಟ", "ಬೂತಕೋಲ"],
        "Happy": ["ನಗುವು", "ಸಂತೋಷ", "ಹರ್ಷ", "ಆನಂದ", "ಉಲ್ಲಾಸ", "ಮೋಹಕ", "ಸಂತೃಪ್ತಿ", "ಹರ್ಷೋಲ್ಲಾಸ", "ಉತ್ಸವ", "ಸುವಾಸನೆ", "ಶ್ರದ್ಧೆ", "ಪ್ರಸನ್ನತೆ", "ಪ್ರಕೃತಿ", "ಶಕ್ತಿಮಂತ್ರ", "ಆಲಾಪನೆ", "ಆಹ್ಲಾದ", "ಚಿರಂಜೀವಿ", "ಜೋಶ್", "ಸ್ಫೂರ್ತಿ", "ಮೋಜು", "ಹಾಸ್ಯ", "ಪ್ರಜ್ಞೆ", "ಸಂಬ್ರಮ", "ಆಕರ್ಷಣ", "ಮಹೋತ್ಸವ", "ಮಹಿಮಾ", "ಪ್ರಾರ್ಥನೆ", "ಆನಂದಿತ", "ಮೋಜುಗಾಡು", "ಚಲನೆ", "ಮೋಹನೆ", "ಅಮೃತ", "ಆನಂದಧಾರೆ", "ಹಬ್ಬದಚಟುವಟಿಕೆಗಳು", "ಸಂಗೀತ", "ಜಶ್ನ್", "ಅದ್ಭುತ", "ವಿಶೇಷ", "ಹೃತ್ಪೂರ್ವಕ", "ಆರಂಭ", "ಆನಂದೋತ್ಸವ", "ಪ್ರಸನ್ನಮುಖ", "ಸುಖಪೂರ್ಣ", "ಹಾಸ್ಯಚಟುವಟಿಕೆ", "ಸ್ನೇಹಪೂರ್ಣ", "ಚಿರಪರಿಚಿತ", "ಪ್ರೋತ್ಸಾಹ", "ಸಿರಿವಂತಿಕೆ", "ಸುದೀರ್ಘ"],
        "Sad": ["ದುಃಖ", "ಕಣ್ಣೀರು", "ನೋವು", "ಬಾಳು", "ವಿಚಾರ", "ಕಳವಳ", "ಮರುಳು", "ಪೀಡೆ", "ಆರತಿ", "ವಿಪತ್ತು", "ತೀವ್ರ ನೋವು", "ಬಿರುಗಾಳಿ", "ಮರಣ", "ಆಲೋಚನೆ", "ಪರಿಶುದ್ಧತೆ", "ಬೇಸರ", "ಪಾತಕ", "ನಷ್ಟ", "ವಿರಹ", "ಆರಾಧನೆ", "ಆಶ್ರು", "ಬದ್ಧ", "ಮೋಹಪಾಶ", "ಅನಾಥ", "ವಿದಾಯ", "ಆಳ್ವಿಕೆ", "ಅಪಘಾತ", "ಶೋಕ", "ಅಪರಾಧ ಭಾವನೆ", "ಪತಿತ", "ಪತನ", "ಹೃದಯವೇದನೆ", "ವಿಚಲನೆ", "ಆರಂಬವೇದನೆ", "ಅಪಮಾನ", "ಅಸಹಾಯಕತೆ", "ಬಿರುಗಾಳಿ", "ಅಪಶ್ರಯ", "ಹೀನತೆಗೆ", "ದ್ವೇಷ", "ಅಧ್ಯಾತ್ಮ ಪೀಡೆ", "ಮರಮಾತು", "ಅಪಸ್ಮೃತಿ", "ಅನಿರೀಕ್ಷಿತ ಪೀಡೆ", "ಹೀನಗತಿ", "ಮೂಢತನ", "ಅಪಮಾನಾತ್ಮಕ", "ಬಿದಿರಹಿತ", "ಹೀನಭಾವ"],
        "Fear": ["ಭಯ", "ಗಾಬರಿ", "ಅಜ್ಞಾನ", "ವಿಕಂಪ", "ಭೀತಿ", "ಆತಂಕ", "ಹೆದರಿಕೆ", "ಭ್ರಾಂತಿ", "ಕಂಪನ", "ಅಪಾಯ", "ಆಘಾತ", "ಅಪಘಾತ", "ಅಪರೂಪ", "ಹಿಡಿದುಕೊಳ್ಳು", "ಅಪರಿಚಿತ", "ಪೀಡೆ", "ಅಪರಾಧ", "ಆಸ್ಪದ", "ಬಿಥ್", "ಅಪಾಯಕಾರಿ", "ಆಕಸ್ಮಿಕ", "ಅಪಾಯಕಾರಕ", "ಅಪಾಯಕಾರಿ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ", "ಅಪಹರಣ", "ಅಪಹಾಸ್ಯ", "ಅನಿಶ್ಚಿತತೆ", "ಅಪಸ್ವರ", "ಅಪಪ್ರಚಾರ", "ಅಪಾಧರಿತ", "ಅಪರಿಚಿತ", "ಅಪಾಯಸೂಚಕ", "ಹಿಂಸೆ", "ಅಪಲೋಚನೆ", "ಅಪರೂಪವಾದ", "ಅಪರಾಧಿ", "ಅಪಮಾನದಾಯಕ", "ಅಪಾಯಕಾರಿ ದಾರಿ", "ಅಪಾಯದ ಕ್ಷಣ", "ಭೀತಿಗೊಳಗಾಗು", "ಆತಂಕಭರಿತ", "ಆಘಾತ ಭರಿತ", "ಅಪಮಾನಿತ", "ಭಯಾನಕ ದೃಶ್ಯ", "ಅಪಯಶಸ್ವಿ", "ಅಪಾಯಕಾರಕ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ ಘಟನೆ", "ಅಪಹೃತ"],
        "Love": ["ಪ್ರೇಮ", "ಮನಸೆ", "ನೀನು", "ನನ್ನ", "ಪ್ರೀತಿ", "ಸ್ನೇಹ", "ಅನುರಾಗ", "ಆಕರ್ಷಣೆ", "ಮಧುರ", "ಅಭಿಮಾನ", "ರೋಮ್ಯಾಂಸ್", "ವೈಭವ", "ಸ್ನೇಹಭಾವ", "ಮೋಹಿತ", "ಪ್ರೀತಿ ಕಥೆ", "ಭಾವನೆ", "ನಟಸಾರ", "ಸಂದೇಶ", "ಆರಾಧನೆ", "ಚರಿತ್ರೆ", "ಸಂಬಂಧ", "ಪತ್ನಿ", "ಪತಿ", "ಪ್ರಿಯ", "ಪ್ರಣಯ", "ಪರಸ್ಪರ", "ಆಕರ್ಷಣ", "ವಲಯ", "ಪರಿಣಯ", "ಹೃದಯ", "ಹೃದಯಸ್ಪರ್ಶಿ", "ಚಿರಸ್ಮರಣೆ", "ಮಾಧುರ್ಯ", "ಅರ್ಥಪೂರ್ಣ", "ಮುದಿತ", "ಮಧುರತೆಯುಳ್ಳ", "ಪ್ರೀತಿಯೆಪ್ರೀತಿ", "ಅಡಿಗುಡಿಯಾಗು", "ಪರಿಪೂರ್ಣತೆ", "ಪವಿತ್ರತೆ", "ಹೃದಯಚಟುಪುಟಿ", "ಮಾಧುರ್ಯಭಾವನೆ", "ಆಧ್ಯಾತ್ಮಿಕ ಪ್ರೇಮ", "ಸ್ನೇಹಪೂರ್ಣ ಸಂಬಂಧ", "ಅಳಭಾವನೆ", "ಆತ್ಮೀಯತೆ", "ಅನಂತ ಪ್ರೇಮ", "ಮಧುರಸಂಭಾಷಣೆ", "ಆಸಕ್ತಿ"]
    }

    rasa_scores = {rasa: 0 for rasa in rasa_keywords}
    for rasa, keywords in rasa_keywords.items():
        for word in keywords:
            rasa_scores[rasa] += lyrics.count(word)

    max_rasa = max(rasa_scores, key=rasa_scores.get)
    return max_rasa if rasa_scores[max_rasa] > 0 else "Unknown"

# Function to scrape lyrics
def get_lyrics(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        lyrics_div = soup.find("div", class_="post-body")

        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
            return lyrics  # Return full lyrics instead of first 6-7 lines

        return "Lyrics not found"
    except Exception as e:
        return f"Error: {e}"

# Extract lyrics and classify
lyrics_data = []
for url in song_links:
    lyrics = get_lyrics(url)
    rasa = classify_rasa(lyrics)
    lyrics_data.append({"URL": url, "Lyrics": lyrics, "Rasa": rasa})

# Save to Excel
df = pd.DataFrame(lyrics_data)
df.to_excel("Kannada_Song_Classification.xlsx", index=False)
print("Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx")
from google.colab import files
files.download("Kannada_Song_Classification.xlsx")



Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of song URLs
song_links = [
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%8E%E0%B2%95%E0%B3%8D%E0%B2%B8%E0%B3%8D%20%E0%B2%95%E0%B3%8D%E0%B2%AF%E0%B3%82%E0%B2%B8%E0%B3%8D%20%E0%B2%AE%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%87%E0%B2%82%E0%B2%A4%E0%B2%BF%20%E0%B2%A8%E0%B2%BF%E0%B2%A8%E0%B3%8D%E0%B2%A8%20%E0%B2%AA%E0%B3%8D%E0%B2%B0%E0%B3%80%E0%B2%A4%E0%B2%BF%E0%B2%AF?&max-results=5",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_89.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9093.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_62.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-10T21:23:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5026.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_56.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6583.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9524.html"
"https://sahityasudhe.blogspot.com/2012/01/1981_6968.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B3%8D%E0%B2%B0%E0%B2%B6%E0%B3%87%E0%B2%95%E0%B2%B0%E0%B3%8D%20%E0%B2%AA%E0%B2%BE%E0%B2%9F%E0%B3%80%E0%B2%B2%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2412.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6865.html",
"https://sahityasudhe.blogspot.com/2012/06/1992_08.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_677.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5098.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2542.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_65.html",
"https://sahityasudhe.blogspot.com/2013/01/2002_9.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3275.html",
"https://sahityasudhe.blogspot.com/2012/06/1997.html",
"https://sahityasudhe.blogspot.com/p/blog-page_6.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_14.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_76.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_80.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9817.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B3%8D%E0%B2%B0%E0%B3%80%E0%B2%A4%E0%B3%8D%E0%B2%B8%E0%B3%87?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5816.html",
"https://sahityasudhe.blogspot.com/2014/03/blog-post_16.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2356.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_50.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4350.html",
"https://sahityasudhe.blogspot.com/2017/09/blog-post_26.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2951.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_65.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6583.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_6968.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%83%E0%B2%A6%E0%B2%AF%20%E0%B2%AA%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B2%B5%E0%B2%BF?updated-max=2012-01-23T19:15:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_195.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%A8%E0%B3%8D%E0%B2%AE%20%E0%B2%9C%E0%B2%A8%E0%B3%8D%E0%B2%AE%E0%B2%A6%20%E0%B2%85%E0%B2%A8%E0%B3%81%E0%B2%AC%E0%B2%82%E0%B2%A7?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu_40.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5395.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_143.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9561.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_36.html",
"https://sahityasudhe.blogspot.com/2012/04/1984_25.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_677.html",
"https://sahityasudhe.blogspot.com/2012/01/2008.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3804.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2975.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B2%BE%E0%B2%97%E0%B3%86%20%E0%B2%B8%E0%B3%81%E0%B2%AE%E0%B3%8D%E0%B2%AE%E0%B2%A8%E0%B3%86?updated-max=2012-01-20T08:09:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_4949.html",
"https://sahityasudhe.blogspot.com/2021/01/kgf-chapter-1.html",
"https://sahityasudhe.blogspot.com/2012/01/1963.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6992.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8571.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_76.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_7802.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_3.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3494.html",
"https://sahityasudhe.blogspot.com/2012/01/2011.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9817.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B3%88%E0%B2%95%E0%B3%8B?&max-results=5",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_4033.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5085.html",
"https://sahityasudhe.blogspot.com/2012/11/1979.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-28T11:51:00%2B05:30&max-results=20&reverse-paginate=true&start=9&by-date=false",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-20T08:09:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2017/09/thili-sanjeleega-operation-alamelamma.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2402.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%86%E0%B2%9F%E0%B3%8B%E0%B2%B0%E0%B2%BE%E0%B2%9C?updated-max=2012-01-21T08:00:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_7008.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2951.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5816.html",
"https://sahityasudhe.blogspot.com/2012/04/2009.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5348.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3452.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%93%E0%B2%82?updated-max=2012-01-21T08:05:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7754.html",
"https://sahityasudhe.blogspot.com/2012/06/1996_4645.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4263.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_704.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B2%AE%E0%B3%8D%E0%B2%AE%E0%B3%82%E0%B2%B0%20%E0%B2%AE%E0%B2%82%E0%B2%A6%E0%B2%BE%E0%B2%B0%20%E0%B2%B9%E0%B3%82%E0%B2%B5%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/04/blog-post_8879.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3797.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B3%86.%20%E0%B2%8E%E0%B2%B8%E0%B3%8D.%20%E0%B2%A8%E0%B2%BF%E0%B2%B8%E0%B3%8D%E0%B2%B8%E0%B2%BE%E0%B2%B0%E0%B3%8D%20%E0%B2%85%E0%B2%B9%E0%B3%8D%E0%B2%AE%E0%B3%86%E0%B2%A6%E0%B3%8D?updated-max=2013-01-18T21:37:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_0.html",
"https://sahityasudhe.blogspot.com/2012/01/1965.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4973.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8805.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A7%E0%B3%8D%E0%B2%B0%E0%B3%81%E0%B2%B5%20%E0%B2%A4%E0%B2%BE%E0%B2%B0%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2402.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_604.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5483.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_574.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_66.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_773.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3638.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9344.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5395.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_143.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B3%86%E0%B2%82%E0%B2%A1%E0%B2%B8%E0%B2%82%E0%B2%AA%E0%B2%BF%E0%B2%97%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2014/03/blog-post.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9561.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1084.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_9751.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1249.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5543.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_8.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5085.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7728.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-09-06T10:12:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1990_5056.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8.%20%E0%B2%B6%E0%B2%BF.%20%E0%B2%AE%E0%B2%B0%E0%B3%81%E0%B2%B3%E0%B2%AF%E0%B3%8D%E0%B2%AF?updated-max=2013-01-16T07:57:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_15.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_27.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7755.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2402.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8098.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2609.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_27.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1084.html"
"https://sahityasudhe.blogspot.com/2012/04/2009.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_14.html",
"https://sahityasudhe.blogspot.com/2017/09/gajavadana-beduve.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3377.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_24.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A1%E0%B2%BF.%20%E0%B2%B8%E0%B2%BF.%20%E0%B2%95%E0%B3%81%E0%B2%B2%E0%B2%95%E0%B2%B0%E0%B3%8D%E0%B2%A3%E0%B2%BF?updated-max=2013-01-16T07:59:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2737.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_8768.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_76.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_47.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1083.html",
"https://sahityasudhe.blogspot.com/2012/01/1964.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AC%E0%B2%B8%E0%B2%B5%E0%B2%A3%E0%B3%8D%E0%B2%A3?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_218.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8.%20%E0%B2%B6%E0%B2%BF.%20%E0%B2%AE%E0%B2%B0%E0%B3%81%E0%B2%B3%E0%B2%AF%E0%B3%8D%E0%B2%AF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3811.html",
"https://sahityasudhe.blogspot.com/2013/01/1996.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%AD%E0%B2%AF%E0%B3%8D?updated-max=2012-01-21T08:16:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%97%E0%B3%86%E0%B2%B3%E0%B3%86%E0%B2%AF?updated-max=2012-01-20T08:11:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2693.html",
"https://sahityasudhe.blogspot.com/2012/01/1988.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2756.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_604.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_61.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_66.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AD%E0%B2%BE%E0%B2%B5%E0%B2%97%E0%B3%80%E0%B2%A4%E0%B3%86?updated-max=2014-02-04T23:15:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_25.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%BF%E0%B2%B0%E0%B2%BF%E0%B2%95%E0%B3%8D%20%E0%B2%AA%E0%B2%BE%E0%B2%B0%E0%B3%8D%E0%B2%9F%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4182.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1449.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2059.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_444.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7100.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_8852.html"
"https://sahityasudhe.blogspot.com/2013/01/blog-post_704.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B3%8D%E0%B2%B5%E0%B2%BE%E0%B2%A4%E0%B2%BF%20%E0%B2%AE%E0%B3%81%E0%B2%A4%E0%B3%8D%E0%B2%A4%E0%B3%81?updated-max=2019-07-13T15:12:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/1985.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3306.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_18.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6769.html",
"https://sahityasudhe.blogspot.com/2012/01/2006.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-28T12:02:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A7%E0%B2%BE%E0%B2%B0%E0%B2%BE%E0%B2%B5%E0%B2%BE%E0%B2%B9%E0%B2%BF?updated-max=2020-08-25T09:02:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5659.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_39.html",
"https://sahityasudhe.blogspot.com/2012/06/1996_2729.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7802.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AD%E0%B2%95%E0%B3%8D%E0%B2%A4%E0%B2%BF%E0%B2%97%E0%B3%80%E0%B2%A4%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/1965.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2645.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%86%E0%B2%A8%E0%B2%82%E0%B2%A6%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4973.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B3%81%E0%B2%B0%E0%B2%82%E0%B2%A6%E0%B2%B0%20%E0%B2%A6%E0%B2%BE%E0%B2%B8?updated-max=2013-01-18T21:35:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8939.html",
"https://sahityasudhe.blogspot.com/2017/09/bhagyada-lakshmi-baramma.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%8A%E0%B2%82%E0%B2%AC%E0%B2%BF%E0%B2%B8%E0%B2%BF%E0%B2%B2%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2985.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%86%E0%B2%95%E0%B2%BE%E0%B2%B6%E0%B3%8D?updated-max=2012-01-21T07:56:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B5%E0%B2%BF%E0%B2%9C%E0%B2%AF%E0%B2%A8%E0%B2%BE%E0%B2%B0%E0%B2%B8%E0%B2%BF%E0%B2%82%E0%B2%B9?updated-max=2013-01-10T21:41:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2743.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B2%E0%B3%81%E0%B2%B8%E0%B2%BF%E0%B2%AF%E0%B2%BE?updated-max=2012-11-06T20:07:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%97%E0%B2%BE%E0%B2%A8%20%E0%B2%AC%E0%B2%9C%E0%B2%BE%E0%B2%A8?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8921.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8426.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2412.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4182.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-01-21T08:02:00%2B05:30&max-results=20&start=422&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/2006_243.html",
"https://sahityasudhe.blogspot.com/2012/06/2006.html",
"https://sahityasudhe.blogspot.com/2013/01/1997.html",
"https://sahityasudhe.blogspot.com/2012/01/1963.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_58.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%B5%E0%B2%BF%E0%B2%B0%E0%B2%A4%E0%B3%8D%E0%B2%A8%20%E0%B2%95%E0%B2%BE%E0%B2%B3%E0%B2%BF%E0%B2%A6%E0%B2%BE%E0%B2%B8?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1083.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7931.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%A3%E0%B3%8D%E0%B2%A3%E0%B2%BF%E0%B2%A8%20%E0%B2%A6%E0%B3%8B%E0%B2%A3%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%8E%E0%B2%AE%E0%B3%8D.%20%E0%B2%8E%E0%B2%A8%E0%B3%8D.%20%E0%B2%B5%E0%B3%8D%E0%B2%AF%E0%B2%BE%E0%B2%B8%20%E0%B2%B0%E0%B2%BE%E0%B2%B5%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4182.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_16.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_9751.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_937.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8910.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2886.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_70.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_5922.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2758.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2475.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9424.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_27.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_50.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2333.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2851.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8805.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2609.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_325.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2886.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%A8%E0%B3%81%E0%B2%B0%E0%B2%BE%E0%B2%97%20%E0%B2%85%E0%B2%B0%E0%B2%B3%E0%B2%BF%E0%B2%A4%E0%B3%81?updated-max=2012-01-23T19:23:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/06/2006_12.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_90.html",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu.html",
"https://sahityasudhe.blogspot.com/2017/09/nammamma-sharade.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_39.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1635.html",
"https://sahityasudhe.blogspot.com/2017/09/gajavadana-beduve.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%B6%E0%B3%8D%E0%B2%B5%E0%B2%AE%E0%B3%87%E0%B2%A7?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A4%E0%B2%BE%E0%B2%9C%E0%B3%8D%20%E0%B2%AE%E0%B2%B9%E0%B2%B2%E0%B3%8D?updated-max=2012-01-20T08:01:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2985.html",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2954.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_63.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2019-07-25T16:16:00%2B05:30&max-results=10",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8426.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%A8%E0%B2%B8%E0%B2%B2%E0%B3%82%20%E0%B2%A8%E0%B3%80%E0%B2%A8%E0%B3%86%20%E0%B2%AE%E0%B2%A8%E0%B2%B8%E0%B2%B2%E0%B3%81%20%E0%B2%A8%E0%B3%80%E0%B2%A8%E0%B3%86?&max-results=5",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_49.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_25.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7100.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%85%E0%B2%82%E0%B2%A4?updated-max=2012-01-23T19:21:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_89.html",
"https://sahityasudhe.blogspot.com/2013/01/2011_9.html",
"https://sahityasudhe.blogspot.com/2012/01/1963.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_771.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_58.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7024.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7132.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2737.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3948.html",
"https://sahityasudhe.blogspot.com/2012/01/paramatma.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_16.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8141.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_5543.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_97.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8910.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post.html",
"https://sahityasudhe.blogspot.com/2012/06/2004_7130.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4464.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B5%E0%B2%BF.%20%E0%B2%A8%E0%B2%BE%E0%B2%97%E0%B3%87%E0%B2%82%E0%B2%A6%E0%B3%8D%E0%B2%B0%20%E0%B2%AA%E0%B3%8D%E0%B2%B0%E0%B2%B8%E0%B2%BE%E0%B2%A6%E0%B3%8D?updated-max=2013-01-18T21:29:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8910.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7024.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%B0%E0%B2%AA%E0%B2%82%E0%B2%9A?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2541.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_79.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2609.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_325.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8805.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_937.html",
"https://sahityasudhe.blogspot.com/2012/06/2006_12.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1306.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A8%E0%B2%B5%E0%B2%97%E0%B3%8D%E0%B2%B0%E0%B2%B9?updated-max=2012-01-20T08:10:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_91.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%BF.%E0%B2%B2%E0%B2%82%E0%B2%95%E0%B3%87%E0%B2%B6?updated-max=2014-03-16T15:21:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_370.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_90.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2743.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BF%E0%B2%AE%E0%B3%8D%E0%B2%AE%E0%B2%BF%20%E0%B2%97%E0%B2%B2%E0%B3%8D%E0%B2%B2%E0%B3%81?updated-max=2013-01-09T21:28:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_574.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B3%81%E0%B2%B2%E0%B2%B5%E0%B2%A7%E0%B3%81?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/2011_143.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_10.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9090.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%83%E0%B2%A6%E0%B2%AF%E0%B2%B6%E0%B2%BF%E0%B2%B5?&max-results=5",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9581.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%82%E0%B2%97%E0%B3%8D%E0%B2%B2%E0%B2%BF?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/09/nammamma-sharade.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7796.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4059.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2954.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-16T07:58:00%2B05:30&max-results=20&start=202&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_63.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_70.html",
"https://sahityasudhe.blogspot.com/2017/09/satyavantara-sangaviralu.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_94.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_9473.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9102.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6378.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_6449.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1089.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%BF.%20%E0%B2%8E%E0%B2%B8%E0%B3%8D.%20%E0%B2%B6%E0%B2%BF%E0%B2%B5%E0%B2%B0%E0%B3%81%E0%B2%A6%E0%B3%8D%E0%B2%B0%E0%B2%AA%E0%B3%8D%E0%B2%AA?&max-results=5",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B2%B5%E0%B2%B3%E0%B3%8D%E0%B2%B3%E0%B2%BF%E0%B2%AF%20%E0%B2%A4%E0%B3%8B%E0%B2%9F?updated-max=2012-11-06T20:11:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5044.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_370.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_27.html",
"https://sahityasudhe.blogspot.com/2013/01/2006_4334.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_243.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%AF%E0%B2%B8%E0%B2%BF%E0%B2%82%E0%B2%B9?updated-max=2013-01-09T21:26:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/06/1996.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AC%E0%B2%BF.%20%E0%B2%8E%E0%B2%82.%20%E0%B2%B6%E0%B3%8D%E0%B2%B0%E0%B3%80%E0%B2%95%E0%B2%82%E0%B2%A0%E0%B2%AF%E0%B3%8D%E0%B2%AF?updated-max=2013-01-18T21:22:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/1981_3716.html",
"https://sahityasudhe.blogspot.com/2012/01/1990_17.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2645.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_9809.html",
"https://sahityasudhe.blogspot.com/2012/01/1963_3599.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5659.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9581.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8805.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7525.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1635.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%B2%E0%B2%BE%E0%B2%B5%E0%B2%BF%E0%B2%A6?&max-results=5",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_15.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AD%E0%B2%95%E0%B3%8D%E0%B2%A4%E0%B2%BF%E0%B2%97%E0%B3%80%E0%B2%A4%E0%B3%86?updated-max=2017-09-06T10:15:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_48.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5439.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2743.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8884.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2462.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8939.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%95%E0%B2%A8%E0%B2%95%20%E0%B2%A6%E0%B2%BE%E0%B2%B8%E0%B2%B0%E0%B3%81?updated-max=2012-01-20T08:07:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7382.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_10.html",
"https://sahityasudhe.blogspot.com/2012/01/1986_17.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_6279.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_9631.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-10T21:23:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/search?updated-max=2012-04-25T23:18:00%2B05:30&max-results=20&reverse-paginate=true",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_4919.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_27.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%A6%E0%B3%8A%E0%B2%A1%E0%B3%8D%E0%B2%A1%E0%B2%B0%E0%B2%82%E0%B2%97%E0%B3%86%20%E0%B2%97%E0%B3%8C%E0%B2%A1?updated-max=2013-01-18T21:31:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1862.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_70.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_3452.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4201.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7382.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8483.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2263.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_94.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AF%E0%B3%81%E0%B2%97%E0%B2%AA%E0%B3%81%E0%B2%B0%E0%B3%81%E0%B2%B7?updated-max=2012-01-21T07:57:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/09/krishna-nee-begane-baaro.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_98.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5218.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_72.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_20.html",
"https://sahityasudhe.blogspot.com/2012/11/2012_6.html",
"https://sahityasudhe.blogspot.com/2012/01/1981.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%87%E0%B2%AC%E0%B3%8D%E0%B2%AC%E0%B2%A8%E0%B2%BF%20%E0%B2%95%E0%B2%B0%E0%B2%97%E0%B2%BF%E0%B2%A4%E0%B3%81?updated-max=2012-01-23T19:18:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_3.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_442.html",
"https://sahityasudhe.blogspot.com/2012/01/2011_9704.html",
"https://sahityasudhe.blogspot.com/2017/09/rathnan-padagalu_40.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AA%E0%B2%B0%E0%B2%AE%E0%B2%BE%E0%B2%A4%E0%B3%8D%E0%B2%AE?&max-results=5",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_44.html",
"https://sahityasudhe.blogspot.com/2012/01/2006_243.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_5218.html",
"https://sahityasudhe.blogspot.com/2017/08/kirik-party-belageddu-lyrics.html",
"https://sahityasudhe.blogspot.com/2012/04/2009_9809.html",
"https://sahityasudhe.blogspot.com/2012/01/1963_3599.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_60.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7423.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_7741.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7525.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2542.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_6378.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1635.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_17.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1879.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1809.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9344.html",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_4.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_2462.html",
"https://sahityasudhe.blogspot.com/search?updated-max=2013-01-09T21:26:00%2B05:30&max-results=20&reverse-paginate=true&start=20&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_4090.html",
"https://sahityasudhe.blogspot.com/2012/01/1982.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%AE%E0%B2%BE%E0%B2%A8%E0%B2%B8%20%E0%B2%B8%E0%B2%B0%E0%B3%8B%E0%B2%B5%E0%B2%B0?updated-max=2012-01-20T07:55:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2019/07/blog-post_82.html",
"https://sahityasudhe.blogspot.com/2012/01/1981_7691.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_2356.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_8420.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_3521.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_7093.html",
"https://sahityasudhe.blogspot.com/2014/02/blog-post_2915.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_2541.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_1476.html",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_9102.html",
"https://sahityasudhe.blogspot.com/2017/08/neenire-saniha-kirik-party-song-lyrics.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B3%82..%E0%B2%9F%E0%B3%8D?&max-results=5",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_243.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B8%E0%B2%82%E0%B2%97%E0%B2%AE?&max-results=5",
"https://sahityasudhe.blogspot.com/search?updated-max=2017-08-24T12:20:00%2B05:30&max-results=20&start=107&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/2011_9.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9A%E0%B2%82%E0%B2%A6%E0%B2%A8%E0%B2%A6%20%E0%B2%97%E0%B3%8A%E0%B2%82%E0%B2%AC%E0%B3%86?updated-max=2012-11-06T20:10:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B9%E0%B3%83%E0%B2%A6%E0%B2%AF%E0%B2%B6%E0%B2%BF%E0%B2%B5?updated-max=2013-01-16T07:53:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2013/01/blog-post_442.html",
"https://sahityasudhe.blogspot.com/2017/08/blog-post_34.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%9C%E0%B2%AF%E0%B2%82%E0%B2%A4%E0%B3%8D%20%E0%B2%95%E0%B2%BE%E0%B2%AF%E0%B3%8D%E0%B2%95%E0%B2%BF%E0%B2%A3%E0%B2%BF?updated-max=2017-08-23T10:23:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_1089.html",
"https://sahityasudhe.blogspot.com/2012/01/blog-post_8527.html",
"https://sahityasudhe.blogspot.com/search/label/%E0%B2%B0%E0%B2%BE%E0%B2%AF%E0%B2%B0%E0%B3%81%20%E0%B2%AC%E0%B2%82%E0%B2%A6%E0%B2%B0%E0%B3%81%20%E0%B2%AE%E0%B2%BE%E0%B2%B5%E0%B2%A8%20%E0%B2%AE%E0%B2%A8%E0%B3%86%E0%B2%97%E0%B3%86?updated-max=2012-01-23T19:16:00%2B05:30&max-results=5&start=5&by-date=false",
"https://sahityasudhe.blogspot.com/2017/08/kirik-party-belageddu-lyrics.html"

]

def classify_rasa(lyrics):
    rasa_keywords = {
        "Devotional": ["ದೇವ", "ಭಕ್ತಿ", "ಪೂಜೆ", "ಹರಿಯೆ", "ಶರಣು", "ರಾಮ", "ಕೃಷ್ಣ", "ವಿಷ್ಣು", "ಪರಮಾತ್ಮ", "ಆರಾಧನೆ", "ಧರ್ಮ", "ಪರಮೇಶ್ವರ", "ಭಕ್ತರು", "ಅರ್ಚನೆ", "ಗೋವಿಂದ", "ಶಿವ", "ದೇವತೆ", "ಅಷ್ಟೋತ್ತರ", "ವಂದನೆ", "ಪೂಜಾರಿ", "ಪವಿತ್ರ", "ಸಂಸ್ಕಾರ", "ಆರಾಧನೆ", "ಸಾಧನೆ", "ಯೋಗ", "ಧ್ಯಾನ", "ಹನುಮಾನ್", "ಪರಮಭಕ್ತ", "ಗುರು", "ದೀಕ್ಷೆ", "ಪ್ರಾರ್ಥನೆ", "ಸಾನ್ನಿಧ್ಯ", "ಶಕ್ತಿ", "ಅಧ್ಯಾತ್ಮ", "ಆತ್ಮದರ್ಶನ", "ಪೂಜಿತ", "ಸಾಗರ", "ಪಾರಮಾರ್ಥ", "ಅವತಾರ", "ಪುನರ್ಜನ್ಮ", "ಬ್ರಹ್ಮ", "ಮೋಕ್ಷ", "ಪುನೀತ", "ಜಪ", "ಸ್ತೋತ್ರ", "ಅಂತ್ಯಕಥೆ", "ಆಧ್ಯಾತ್ಮಿಕ", "ಅಮೃತ", "ಚಕ್ರ"],
        "Patriotic": ["ದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ", "ಜೈಹಿಂದ್", "ಸಮರ್ಪಣೆ", "ಭಾರತ", "ಧ್ವಜ", "ಸ್ವಾಭಿಮಾನ", "ವೀರ", "ರಾಜ್ಯ", "ಏಕತೆ", "ರಾಷ್ಟ್ರ", "ಬಾಳಿದೇಶ", "ಸ್ವಾತಂತ್ರ್ಯ ಹೋರಾಟ", "ಭಾರತ ಮಾತಾ", "ಜೈ ಭಾರತ", "ಭಾರತೀಯ", "ಹೋರಾಟಗಾರ", "ಬಸವಣ್ಣ", "ಸ್ಪರ್ಧೆ", "ವೀರಸೇನೆ", "ಸಮಾನತೆ", "ಪ್ರಜಾಸತ್ತಾತ್ಮಕ", "ಜಾತಿ", "ರಾಷ್ಟ್ರಪ್ರೇಮ", "ಸೈನಿಕ", "ಸಾಮರಸ್ಯ", "ಹಕ್ಕು", "ವಿದ್ಯಾರ್ಥಿ", "ಸಂಘಟನೆ", "ಸಂಭ್ರಮ", "ಬಿಗ್ ಡೇ", "ಅಜಯ್ ಭಾರತ", "ದೇಶಭಕ್ತ", "ಪಾಲ್ಗೊಂಡರು", "ಹೋರಾಟ", "ಆಂದೋಲನ", "ಸಂಬಂಧ", "ಹೈದರಾಬಾದ್", "ಗೌರವ", "ಆತ್ಮವಿಶ್ವಾಸ", "ಧೈರ್ಯ", "ಅಖಂಡ ಭಾರತ", "ಪರಾಕ್ರಮ", "ರಕ್ಷಕ", "ಪರಮವೀರ", "ಪರಿಪೂರ್ಣ", "ಶೌರ್ಯ", "ವಿಜಯ", "ವಿಜಯೋತ್ಸವ"],
        "Folk": ["ಜನಪದ", "ಹೊಲವು", "ಕುಟುಕು", "ಗ್ರಾಮೀಣ", "ಬಾಲಪಾಟ", "ಕುಣಿತ", "ನಾಡು", "ಹಬ್ಬ", "ಹಳ್ಳಿ", "ಕೃತನೆ", "ಹೆಜ್ಜೆ", "ಹೊರಾಟ", "ಬಾವಲಿ", "ದೋಣಿ", "ಮುದ್ದು", "ಬಡವರ", "ಬೂದಿ", "ಮಳಗು", "ಪಾದ", "ಬದನೆ", "ಹಳ್ಳಿಗುಡಿ", "ಕಾಯ್ಕಳು", "ಕಬ್ಬಿಣ", "ಕೃಷಿ", "ಬಟ್ಟಲು", "ಹಾದಿ", "ಬೀದಿ", "ಅಂಗಡಿ", "ರಸಗೆಜೆ", "ಗೊಡ್ಡಲು", "ಅರವಟ್ಟೆ", "ಅಕ್ಕಸಾಲಿಗೆ", "ಆಡುಗೊಳಲು", "ಅಕ್ಕಪಕ್ಕ", "ಹೂವಿನ ಹಾರ", "ಮಡಕೆ", "ಮೂಡಲಗೇರಿ", "ಮಕ್ಕಳಾಟ", "ನೆಲದಾಟ", "ಮಾಲಿಗೆ", "ಚಪ್ಪಲಿ", "ಬಸವಣ", "ದೀಪಾಲಂಕಾರ", "ಗಂಗಾವತಿ", "ಕಂಭಳ", "ಹೆಜ್ಜೆಗುಂಡಿ", "ಮರಹರಕೆ", "ಚಂದ್ರಹಾರ", "ಪಡುಗೆಯಾಟ", "ಬೂತಕೋಲ"],
        "Happy": ["ನಗುವು", "ಸಂತೋಷ", "ಹರ್ಷ", "ಆನಂದ", "ಉಲ್ಲಾಸ", "ಮೋಹಕ", "ಸಂತೃಪ್ತಿ", "ಹರ್ಷೋಲ್ಲಾಸ", "ಉತ್ಸವ", "ಸುವಾಸನೆ", "ಶ್ರದ್ಧೆ", "ಪ್ರಸನ್ನತೆ", "ಪ್ರಕೃತಿ", "ಶಕ್ತಿಮಂತ್ರ", "ಆಲಾಪನೆ", "ಆಹ್ಲಾದ", "ಚಿರಂಜೀವಿ", "ಜೋಶ್", "ಸ್ಫೂರ್ತಿ", "ಮೋಜು", "ಹಾಸ್ಯ", "ಪ್ರಜ್ಞೆ", "ಸಂಬ್ರಮ", "ಆಕರ್ಷಣ", "ಮಹೋತ್ಸವ", "ಮಹಿಮಾ", "ಪ್ರಾರ್ಥನೆ", "ಆನಂದಿತ", "ಮೋಜುಗಾಡು", "ಚಲನೆ", "ಮೋಹನೆ", "ಅಮೃತ", "ಆನಂದಧಾರೆ", "ಹಬ್ಬದಚಟುವಟಿಕೆಗಳು", "ಸಂಗೀತ", "ಜಶ್ನ್", "ಅದ್ಭುತ", "ವಿಶೇಷ", "ಹೃತ್ಪೂರ್ವಕ", "ಆರಂಭ", "ಆನಂದೋತ್ಸವ", "ಪ್ರಸನ್ನಮುಖ", "ಸುಖಪೂರ್ಣ", "ಹಾಸ್ಯಚಟುವಟಿಕೆ", "ಸ್ನೇಹಪೂರ್ಣ", "ಚಿರಪರಿಚಿತ", "ಪ್ರೋತ್ಸಾಹ", "ಸಿರಿವಂತಿಕೆ", "ಸುದೀರ್ಘ"],
        "Sad": ["ದುಃಖ", "ಕಣ್ಣೀರು", "ನೋವು", "ಬಾಳು", "ವಿಚಾರ", "ಕಳವಳ", "ಮರುಳು", "ಪೀಡೆ", "ಆರತಿ", "ವಿಪತ್ತು", "ತೀವ್ರ ನೋವು", "ಬಿರುಗಾಳಿ", "ಮರಣ", "ಆಲೋಚನೆ", "ಪರಿಶುದ್ಧತೆ", "ಬೇಸರ", "ಪಾತಕ", "ನಷ್ಟ", "ವಿರಹ", "ಆರಾಧನೆ", "ಆಶ್ರು", "ಬದ್ಧ", "ಮೋಹಪಾಶ", "ಅನಾಥ", "ವಿದಾಯ", "ಆಳ್ವಿಕೆ", "ಅಪಘಾತ", "ಶೋಕ", "ಅಪರಾಧ ಭಾವನೆ", "ಪತಿತ", "ಪತನ", "ಹೃದಯವೇದನೆ", "ವಿಚಲನೆ", "ಆರಂಬವೇದನೆ", "ಅಪಮಾನ", "ಅಸಹಾಯಕತೆ", "ಬಿರುಗಾಳಿ", "ಅಪಶ್ರಯ", "ಹೀನತೆಗೆ", "ದ್ವೇಷ", "ಅಧ್ಯಾತ್ಮ ಪೀಡೆ", "ಮರಮಾತು", "ಅಪಸ್ಮೃತಿ", "ಅನಿರೀಕ್ಷಿತ ಪೀಡೆ", "ಹೀನಗತಿ", "ಮೂಢತನ", "ಅಪಮಾನಾತ್ಮಕ", "ಬಿದಿರಹಿತ", "ಹೀನಭಾವ"],
        "Fear": ["ಭಯ", "ಗಾಬರಿ", "ಅಜ್ಞಾನ", "ವಿಕಂಪ", "ಭೀತಿ", "ಆತಂಕ", "ಹೆದರಿಕೆ", "ಭ್ರಾಂತಿ", "ಕಂಪನ", "ಅಪಾಯ", "ಆಘಾತ", "ಅಪಘಾತ", "ಅಪರೂಪ", "ಹಿಡಿದುಕೊಳ್ಳು", "ಅಪರಿಚಿತ", "ಪೀಡೆ", "ಅಪರಾಧ", "ಆಸ್ಪದ", "ಬಿಥ್", "ಅಪಾಯಕಾರಿ", "ಆಕಸ್ಮಿಕ", "ಅಪಾಯಕಾರಕ", "ಅಪಾಯಕಾರಿ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ", "ಅಪಹರಣ", "ಅಪಹಾಸ್ಯ", "ಅನಿಶ್ಚಿತತೆ", "ಅಪಸ್ವರ", "ಅಪಪ್ರಚಾರ", "ಅಪಾಧರಿತ", "ಅಪರಿಚಿತ", "ಅಪಾಯಸೂಚಕ", "ಹಿಂಸೆ", "ಅಪಲೋಚನೆ", "ಅಪರೂಪವಾದ", "ಅಪರಾಧಿ", "ಅಪಮಾನದಾಯಕ", "ಅಪಾಯಕಾರಿ ದಾರಿ", "ಅಪಾಯದ ಕ್ಷಣ", "ಭೀತಿಗೊಳಗಾಗು", "ಆತಂಕಭರಿತ", "ಆಘಾತ ಭರಿತ", "ಅಪಮಾನಿತ", "ಭಯಾನಕ ದೃಶ್ಯ", "ಅಪಯಶಸ್ವಿ", "ಅಪಾಯಕಾರಕ ಪರಿಸ್ಥಿತಿ", "ಆಘಾತಕಾರಿ ಘಟನೆ", "ಅಪಹೃತ"],
        "Love": ["ಪ್ರೇಮ", "ಮನಸೆ", "ನೀನು", "ನನ್ನ", "ಪ್ರೀತಿ", "ಸ್ನೇಹ", "ಅನುರಾಗ", "ಆಕರ್ಷಣೆ", "ಮಧುರ", "ಅಭಿಮಾನ", "ರೋಮ್ಯಾಂಸ್", "ವೈಭವ", "ಸ್ನೇಹಭಾವ", "ಮೋಹಿತ", "ಪ್ರೀತಿ ಕಥೆ", "ಭಾವನೆ", "ನಟಸಾರ", "ಸಂದೇಶ", "ಆರಾಧನೆ", "ಚರಿತ್ರೆ", "ಸಂಬಂಧ", "ಪತ್ನಿ", "ಪತಿ", "ಪ್ರಿಯ", "ಪ್ರಣಯ", "ಪರಸ್ಪರ", "ಆಕರ್ಷಣ", "ವಲಯ", "ಪರಿಣಯ", "ಹೃದಯ", "ಹೃದಯಸ್ಪರ್ಶಿ", "ಚಿರಸ್ಮರಣೆ", "ಮಾಧುರ್ಯ", "ಅರ್ಥಪೂರ್ಣ", "ಮುದಿತ", "ಮಧುರತೆಯುಳ್ಳ", "ಪ್ರೀತಿಯೆಪ್ರೀತಿ", "ಅಡಿಗುಡಿಯಾಗು", "ಪರಿಪೂರ್ಣತೆ", "ಪವಿತ್ರತೆ", "ಹೃದಯಚಟುಪುಟಿ", "ಮಾಧುರ್ಯಭಾವನೆ", "ಆಧ್ಯಾತ್ಮಿಕ ಪ್ರೇಮ", "ಸ್ನೇಹಪೂರ್ಣ ಸಂಬಂಧ", "ಅಳಭಾವನೆ", "ಆತ್ಮೀಯತೆ", "ಅನಂತ ಪ್ರೇಮ", "ಮಧುರಸಂಭಾಷಣೆ", "ಆಸಕ್ತಿ"]
    }

    rasa_scores = {rasa: 0 for rasa in rasa_keywords}
    for rasa, keywords in rasa_keywords.items():
        for word in keywords:
            rasa_scores[rasa] += lyrics.count(word)

    max_rasa = max(rasa_scores, key=rasa_scores.get)
    return max_rasa if rasa_scores[max_rasa] > 0 else "Unknown"

# Function to scrape lyrics
def get_lyrics(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        lyrics_div = soup.find("div", class_="post-body")

        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
            return lyrics  # Return full lyrics instead of first 6-7 lines

        return "Lyrics not found"
    except Exception as e:
        return f"Error: {e}"

# Extract lyrics and classify
lyrics_data = []
for url in song_links:
    lyrics = get_lyrics(url)
    rasa = classify_rasa(lyrics)
    lyrics_data.append({"URL": url, "Lyrics": lyrics, "Rasa": rasa})

# Save to Excel
df = pd.DataFrame(lyrics_data)
df.to_excel("Kannada_Song_Classification.xlsx", index=False)
print("Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx")
from google.colab import files
files.download("Kannada_Song_Classification.xlsx")


Lyrics extraction and classification completed! Check Kannada_Song_Classification.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Load dataset
file_path = "/content/Kannada_Song_Classification (7).xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")
df

,URL,Lyrics,Rasa
0,https://sahityasudhe.blogspot.com/search/label...,ರೋಡಿಗಿಳಿ ರಾಧಿಕ ಚಿಂದಿ ಮಾಡೇ ಚಂದ್ರಿಕಾ\n\n\n\n ಚಿ...,Unknown
1,https://sahityasudhe.blogspot.com/search/label...,ಮಧುವನ ಕರೆದರೆ\n\n\n\nಚಿತ್ರ: ಇಂತಿ ನಿನ್ನ ಪ್ರೀತಿಯ ...,Love
2,https://sahityasudhe.blogspot.com/2019/07/blog...,ಪುಟ್ಟಮಲ್ಲಿ ಪುಟ್ಟಮಲ್ಲಿ ಕೇಳು\n\nಸಂಗೀತ/ಸಾಹಿತ್ಯ: ಹ...,Sad
3,https://sahityasudhe.blogspot.com/2012/01/blog...,"ಚಿತ್ರ: ಬೊಂಬಾಟ್ \nಗಾಯನ: ಸೋನು ನಿಗಮ್, ಶ್ರೇಯ ಗೋಶಲ್...",Love
4,https://sahityasudhe.blogspot.com/2017/08/blog...,ಬಸವಣ್ಣನವರ ವಚನಗಳು : ಭಾಗ - ೫\n\n\n\n\n\n೪೦೧.\n\n...,Devotional
...,...,...,...
1433,https://sahityasudhe.blogspot.com/search/label...,ನಾ ಬಿಡಲಾರೆ ಎಂದು ನಿನ್ನ\n\n\n\nಚಿತ್ರ: ಪ್ರೇಮದ ಕಾಣ...,Love
1434,https://sahityasudhe.blogspot.com/search/label...,ನೀ ನಡೆದರೆ ಸೊಗಸು...\n\n\n\nಚಿತ್ರ: ಅನುರಾಗ ಅರಳಿತು...,Happy
1435,https://sahityasudhe.blogspot.com/2017/09/bhag...,ಭಾಗ್ಯದ ಲಕ್ಷ್ಮಿ ಬಾರಮ್ಮ ನಮ್ಮಮ್ಮಾ ಶ್ರೀ\n\nಸೌಭಾಗ್ಯ...,Devotional
1436,https://sahityasudhe.blogspot.com/2013/01/blog...,ದೀಪವು ನಿನ್ನದೆ...\n\n\n\n- ಕೆ. ಎಸ್. ನರಸಿಂಹಸ್ವಾಮ...,Folk


In [ ]:
# Drop missing values
df = df.dropna(subset=["Lyrics", "Rasa"])

# Text cleaning function
def clean_lyrics(text):
    text = re.sub(r"\n+", " ", text)  # Remove newlines
    text = re.sub(r"[^ಀ-೿a-zA-Z0-9\s]", "", text)  # Keep Kannada and alphanumeric
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

df["Lyrics"] = df["Lyrics"].apply(clean_lyrics)

# Split data into train & test
X_train, X_test, y_train, y_test = train_test_split(df["Lyrics"], df["Rasa"], test_size=0.2, random_state=42)

# Create TF-IDF + SVM pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),  # TF-IDF with unigrams & bigrams
    ('svm', SVC(kernel='linear', probability=True))  # Linear SVM
])

# Train the model
pipeline.fit(X_train, y_train)

# Test accuracy
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Function to predict rasa of a new song
def predict_rasa(lyrics):
    cleaned_lyrics = clean_lyrics(lyrics)
    return pipeline.predict([cleaned_lyrics])[0]

# Example usage
sample_song = "ನಮ್ಮ ದೇಶದ ಧ್ವಜ ಹಾರಲಿ, ಜಯ ಭಾರತ ಮಾತಾ!"  # Example patriotic song
print("Predicted Rasa:", predict_rasa(sample_song))

<ipython-input-2-1805de7bfda5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Lyrics"] = df["Lyrics"].apply(clean_lyrics)


Accuracy: 0.8362369337979094
Predicted Rasa: Devotional


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Load dataset
file_path = "/content/Kannada_Song_Classification (7).xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Drop missing values
df = df.dropna(subset=["Lyrics", "Rasa"])

# Remove 'Unknown' and 'Surprise' rasa
df = df[(df["Rasa"] != "Unknown") & (df["Rasa"] != "Surprise")]

# Count each rasa category
rasa_counts = df['Rasa'].value_counts()
print("Count of each Rasa:")
print(rasa_counts)

# Check class distribution before balancing
print("Class distribution before balancing:")
print(df['Rasa'].value_counts())

# Text cleaning function
def clean_lyrics(text):
    text = re.sub(r"\n+", " ", text)  # Remove newlines
    text = re.sub(r"[^ಀ-೿a-zA-Z0-9\s]", "", text)  # Keep Kannada and alphanumeric
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

df["Lyrics"] = df["Lyrics"].apply(clean_lyrics)

# Split data into train & test
X_train, X_test, y_train, y_test = train_test_split(df["Lyrics"], df["Rasa"], test_size=0.2, random_state=42, stratify=df["Rasa"])

# Encode labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Convert text data to numerical representation
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_tfidf_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Train SVM model
svm_model = SVC(kernel='rbf', probability=True, class_weight='balanced')
svm_model.fit(X_train_tfidf_resampled, y_train_resampled)

# Make predictions
y_pred = svm_model.predict(X_test_tfidf)
y_pred = encoder.inverse_transform(y_pred)  # Convert back to rasa names
y_test_labels = encoder.inverse_transform(y_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print("Classification Report:")
print(classification_report(y_test_labels, y_pred))

# Function to predict rasa of a new song
def predict_rasa(lyrics):
    cleaned_lyrics = clean_lyrics(lyrics)
    transformed_lyrics = vectorizer.transform([cleaned_lyrics])
    predicted_label = svm_model.predict(transformed_lyrics)[0]
    return encoder.inverse_transform([predicted_label])[0]

# Example usage
sample_song = "ನಮ್ಮ ದೇಶದ ಧ್ವಜ ಹಾರಲಿ, ಜಯ ಭಾರತ ಮಾತಾ!"  # Example patriotic song
print("Predicted Rasa:", predict_rasa(sample_song))


Count of each Rasa:
Rasa
Love          587
Devotional    327
Folk           99
Sad            57
Happy          52
Patriotic      33
Fear           11
Name: count, dtype: int64
Class distribution before balancing:
Rasa
Love          587
Devotional    327
Folk           99
Sad            57
Happy          52
Patriotic      33
Fear           11
Name: count, dtype: int64
Accuracy: 0.8247863247863247
Classification Report:
              precision    recall  f1-score   support

  Devotional       0.84      0.79      0.81        66
        Fear       0.40      1.00      0.57         2
        Folk       0.83      0.50      0.62        20
       Happy       0.50      0.60      0.55        10
        Love       0.87      0.92      0.89       118
   Patriotic       1.00      0.86      0.92         7
         Sad       0.69      0.82      0.75        11

    accuracy                           0.82       234
   macro avg       0.73      0.78      0.73       234
weighted avg       0.83      0.82  

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import GridSearchCV

# Load dataset
file_path = "/content/Kannada_Song_Classification (7).xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Drop missing values
df = df.dropna(subset=["Lyrics", "Rasa"])

# Remove 'Unknown' and 'Surprise' rasa
df = df[(df["Rasa"] != "Unknown") & (df["Rasa"] != "Surprise")]

# Count each rasa category
rasa_counts = df['Rasa'].value_counts()
print("Count of each Rasa:")
print(rasa_counts)

# Check class distribution before balancing
print("Class distribution before balancing:")
print(df['Rasa'].value_counts())

# Text cleaning function
def clean_lyrics(text):
    text = re.sub(r"\n+", " ", text)  # Remove newlines
    text = re.sub(r"[^ಀ-೿a-zA-Z0-9\s]", "", text)  # Keep Kannada and alphanumeric
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

df["Lyrics"] = df["Lyrics"].apply(clean_lyrics)

# Split data into train & test
X_train, X_test, y_train, y_test = train_test_split(df["Lyrics"], df["Rasa"], test_size=0.2, random_state=42, stratify=df["Rasa"])

# Encode labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Convert text data to numerical representation
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=20000)  # Increased max_features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Handle class imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_tfidf_resampled, y_train_resampled = adasyn.fit_resample(X_train_tfidf, y_train)

# Hyperparameter tuning for SVM
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['rbf']
}
svm = SVC(probability=True, class_weight='balanced')
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train_tfidf_resampled, y_train_resampled)

# Best model
best_svm = grid_search.best_estimator_

# Make predictions
y_pred = best_svm.predict(X_test_tfidf)
y_pred = encoder.inverse_transform(y_pred)  # Convert back to rasa names
y_test_labels = encoder.inverse_transform(y_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print("Classification Report:")
print(classification_report(y_test_labels, y_pred))

# Function to predict rasa of a new song
def predict_rasa(lyrics):
    cleaned_lyrics = clean_lyrics(lyrics)
    transformed_lyrics = vectorizer.transform([cleaned_lyrics])
    predicted_label = best_svm.predict(transformed_lyrics)[0]
    return encoder.inverse_transform([predicted_label])[0]

# Example usage
sample_song = "ನಮ್ಮ ದೇಶದ ಧ್ವಜ ಹಾರಲಿ, ಜಯ ಭಾರತ ಮಾತಾ!"  # Example patriotic song
print("Predicted Rasa:", predict_rasa(sample_song))


Count of each Rasa:
Rasa
Love          587
Devotional    327
Folk           99
Sad            57
Happy          52
Patriotic      33
Fear           11
Name: count, dtype: int64
Class distribution before balancing:
Rasa
Love          587
Devotional    327
Folk           99
Sad            57
Happy          52
Patriotic      33
Fear           11
Name: count, dtype: int64
Accuracy: 0.8461538461538461
Classification Report:
              precision    recall  f1-score   support

  Devotional       0.84      0.79      0.81        66
        Fear       0.40      1.00      0.57         2
        Folk       0.88      0.70      0.78        20
       Happy       0.67      0.60      0.63        10
        Love       0.89      0.92      0.90       118
   Patriotic       1.00      0.86      0.92         7
         Sad       0.71      0.91      0.80        11

    accuracy                           0.85       234
   macro avg       0.77      0.82      0.77       234
weighted avg       0.85      0.85  

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from collections import Counter

# Load dataset
file_path = "/content/Kannada_Song_Classification (7).xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Drop missing values
df = df.dropna(subset=["Lyrics", "Rasa"])

# Remove 'Unknown' and 'Surprise' rasa
df = df[(df["Rasa"] != "Unknown") & (df["Rasa"] != "Surprise")]

# Text cleaning function
def clean_lyrics(text):
    text = re.sub(r"\n+", " ", text)  # Remove newlines
    text = re.sub(r"[^ಀ-೿a-zA-Z0-9\s]", "", text)  # Keep Kannada and alphanumeric
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

df["Lyrics"] = df["Lyrics"].apply(clean_lyrics)

# Count each rasa category
print("Original Class Distribution:\n", df['Rasa'].value_counts())


Original Class Distribution:
 Rasa
Love          587
Devotional    327
Folk           99
Sad            57
Happy          52
Patriotic      33
Fear           11
Name: count, dtype: int64


In [ ]:
# Balance dataset: Oversample minority & undersample majority
target_count = 300  # Set to 300 per class
majority_classes = ["Love"]  # Classes with more than 300 samples
minority_classes = df['Rasa'].value_counts()[df['Rasa'].value_counts() < target_count].index.tolist()

# Undersample majority classes
df_majority = df[df['Rasa'].isin(majority_classes)].sample(n=target_count, random_state=42)

# Oversample minority classes
df_minority_resampled = df_minority.groupby("Rasa", group_keys=False).apply(
    lambda x: x.sample(n=target_count, replace=True, random_state=42)
).reset_index(drop=True)

<ipython-input-6-897cbb5a5760>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_minority_resampled = df_minority.groupby("Rasa", group_keys=False).apply(


In [ ]:
#Combine balanced dataset
df_balanced = pd.concat([df_majority, df_minority_resampled])
print("Balanced Class Distribution:\n", df_balanced['Rasa'].value_counts())

# Split data into train & test
X_train, X_test, y_train, y_test = train_test_split(
    df_balanced["Lyrics"], df_balanced["Rasa"], test_size=0.2, random_state=42, stratify=df_balanced["Rasa"]
)

# Encode labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


Balanced Class Distribution:
 Rasa
Love         300
Fear         300
Folk         300
Happy        300
Patriotic    300
Sad          300
Name: count, dtype: int64


In [ ]:
# Convert text data to numerical representation
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=15000)  # Reduced max_features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Handle class imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_tfidf_resampled, y_train_resampled = adasyn.fit_resample(X_train_tfidf, y_train)

# Hyperparameter tuning for SVM
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}
svm = SVC(probability=True, class_weight='balanced')
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train_tfidf_resampled, y_train_resampled)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', probability=True),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
             scoring='f1_weighted', verbose=1)

In [ ]:
# Best model
best_svm = grid_search.best_estimator_

# Make predictions
y_pred = best_svm.predict(X_test_tfidf)
y_pred = encoder.inverse_transform(y_pred)  # Convert back to rasa names
y_test_labels = encoder.inverse_transform(y_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print("Classification Report:\n", classification_report(y_test_labels, y_pred))

# Function to predict rasa of a new song
def predict_rasa(lyrics):
    cleaned_lyrics = clean_lyrics(lyrics)
    transformed_lyrics = vectorizer.transform([cleaned_lyrics])
    predicted_label = best_svm.predict(transformed_lyrics)[0]
    return encoder.inverse_transform([predicted_label])[0]

# Example usage
sample_song = "ನಮ್ಮ ದೇಶದ ಧ್ವಜ ಹಾರಲಿ, ಜಯ ಭಾರತ ಮಾತಾ!"  # Example patriotic song
print("Predicted Rasa:", predict_rasa(sample_song))

Accuracy: 0.9638888888888889
Classification Report:
               precision    recall  f1-score   support

        Fear       0.94      1.00      0.97        60
        Folk       0.92      0.90      0.91        60
       Happy       1.00      0.93      0.97        60
        Love       0.97      0.95      0.96        60
   Patriotic       0.98      1.00      0.99        60
         Sad       0.98      1.00      0.99        60

    accuracy                           0.96       360
   macro avg       0.96      0.96      0.96       360
weighted avg       0.96      0.96      0.96       360

Predicted Rasa: Love


In [ ]:
!pip install fasttext
!pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313472 sha256=8139cbabe0d3d4d87d26dc719dba5dca1d49c840c00af6173b3f0b0a291ed76f
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
import pandas as pd
import re
import fasttext
import fasttext.util
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
file_path = "/content/Kannada_Song_Classification (7).xlsx"  # Change the path if needed
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Drop missing values
df = df.dropna(subset=["Lyrics", "Rasa"])

# Remove 'Unknown' and 'Surprise' rasa
df = df[(df["Rasa"] != "Unknown") & (df["Rasa"] != "Surprise")]

# Text cleaning function
def clean_lyrics(text):
    text = re.sub(r"\n+", " ", text)  # Remove newlines
    text = re.sub(r"[^ಀ-೿a-zA-Z0-9\s]", "", text)  # Keep Kannada and alphanumeric
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

df["Lyrics"] = df["Lyrics"].apply(clean_lyrics)

# Balance dataset (Make 300 songs per Rasa)
def balance_dataset(df, target_size=300):
    dfs = []
    for rasa in df["Rasa"].unique():
        rasa_df = df[df["Rasa"] == rasa]
        if len(rasa_df) < target_size:
            rasa_df = rasa_df.sample(target_size, replace=True, random_state=42)
        else:
            rasa_df = rasa_df.sample(target_size, replace=False, random_state=42)
        dfs.append(rasa_df)
    return pd.concat(dfs).reset_index(drop=True)

df = balance_dataset(df, target_size=300)

# Check class distribution after balancing
print("Class distribution after balancing:")
print(df["Rasa"].value_counts())

# Split data into train & test
X_train, X_test, y_train, y_test = train_test_split(df["Lyrics"], df["Rasa"], test_size=0.2, random_state=42, stratify=df["Rasa"])

# Encode labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Load FastText Kannada embeddings
fasttext.util.download_model('kn', if_exists='ignore')  # Download Kannada model
ft = fasttext.load_model('cc.kn.300.bin')

# Function to get FastText vector for a sentence
def get_sentence_vector(text):
    words = text.split()
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(300)

# Convert lyrics to embeddings
X_train_vec = np.array([get_sentence_vector(lyric) for lyric in X_train])
X_test_vec = np.array([get_sentence_vector(lyric) for lyric in X_test])

# Handle class imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

# Hyperparameter tuning for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}
svm = SVC(probability=True, class_weight='balanced')
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train_resampled, y_train_resampled)

# Best model
best_svm = grid_search.best_estimator_

# Make predictions
y_pred = best_svm.predict(X_test_vec)
y_pred = encoder.inverse_transform(y_pred)  # Convert back to rasa names
y_test_labels = encoder.inverse_transform(y_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test_labels, y_pred))
print("Classification Report:")
print(classification_report(y_test_labels, y_pred))

# Confusion matrix visualization
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_test_labels, y_pred), annot=True, fmt="d", cmap="Blues",
            xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Function to predict rasa of a new song
def predict_rasa(lyrics):
    cleaned_lyrics = clean_lyrics(lyrics)
    transformed_lyrics = get_sentence_vector(cleaned_lyrics).reshape(1, -1)
    predicted_label = best_svm.predict(transformed_lyrics)[0]
    return encoder.inverse_transform([predicted_label])[0]

# Example usage
sample_song = "ನಮ್ಮ ದೇಶದ ಧ್ವಜ ಹಾರಲಿ, ಜಯ ಭಾರತ ಮಾತಾ!"  # Example patriotic song
print("Predicted Rasa:", predict_rasa(sample_song))


Class distribution after balancing:
Rasa
Love          300
Sad           300
Devotional    300
Folk          300
Happy         300
Patriotic     300
Fear          300
Name: count, dtype: int64
